In [1]:
#Importar paquetesgroup
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

24-04-2024


In [3]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

In [4]:
print(datetime.now())

#Importar bases de datos
bd_gastos_tr = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'Data_Gastos_TipoRecurso_2018-2024_9Abr2024_2' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_ingresos_tr = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'Data_Ingresos_TipoRecurso_2018-2024_9Abr2024' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_sd = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'SD 20.04.2024 - GR GL' + '.xlsx'), sheet_name='Hoja1', dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_rubro18 = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'SB_Rubro18_06042024' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_R18= pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'CANON_190424_VF_sindecimal - GRGL' + '.xlsx'),  sheet_name='Hoja3', dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_seg23 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'SeguimientoGasto_2023_04032024' + '.csv'))
bd_seg24 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'SeguimientoGasto_2024_'+ fecha_corte +'.csv'))
dict23 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'diccionarios_seguimiento_2023' + '.xlsx'), sheet_name='COD_CADENA_GASTO')
dict24 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'diccionarios_seguimiento_2024' + '.xlsx'), sheet_name='COD_CADENA_GASTO')
dict231 = pd.read_excel(os.path.join(r'Y:Data_CSV\SIAFGasto2023', 'diccionarios_seguimiento_2023' + '.xlsx'), sheet_name='COD_ID')
dict241 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'diccionarios_seguimiento_2024' + '.xlsx'), sheet_name='COD_ID')
print(datetime.now())

2024-04-24 14:35:43.616944
2024-04-24 14:39:56.280554


# 1.Analisis Tipo Recurso PMG 2025 -2027

## Informaciòn Gastos

### Conversión de BD (a nivel de tipo de recurso) - 2018 - 2023

In [5]:
bd_gastos_tr.info()
bd_gastos_tr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107579 entries, 0 to 107578
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ANO_EJE           107579 non-null  int32  
 1   NIVEL_GOB         107579 non-null  object 
 2   SECTOR            107579 non-null  object 
 3   PLIEGO            107579 non-null  object 
 4   UE                107579 non-null  object 
 5   SEC_EJEC          107579 non-null  object 
 6   FUENTE            107579 non-null  object 
 7   RUBRO             107579 non-null  object 
 8   TIPO_RECURSO      107579 non-null  object 
 9   CATEGORIA_GTO     107579 non-null  object 
 10  TIPO_TRANSACCION  107579 non-null  object 
 11  GENERICA          107579 non-null  object 
 12  SUBGENERICA       107579 non-null  object 
 13  SUBGENERICA_DET   107579 non-null  object 
 14  ESPECIFICA        107579 non-null  object 
 15  ESPECIFICA_DET    107579 non-null  object 
 16  PIA               10

,ANO_EJE,NIVEL_GOB,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICACION,COMPROMISO_ANUAL,DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,11. OTROS SERVICIOS,99. SERVICIOS DIVERSOS,0,0,0.0,0.0,211584.99
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,1. DE VEHICULOS,0,0,0.0,0.0,5100.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,"1. SERVICIOS DE CONSULTORIAS, ASESORIAS Y SIMI...",99. OTROS SERVICIOS SIMILARES,0,0,0.0,0.0,8460.00
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,99. COMPRA DE OTROS BIENES,1. COMPRA DE OTROS BIENES,1. HERRAMIENTAS,0,0,0.0,0.0,12773.00
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,11. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. PARA EDIFICIOS Y ESTRUCTURAS,0,0,0.0,0.0,39520.50


In [6]:
hist_gastos = bd_gastos_tr.copy()
hist_gastos.rename(columns={'NIVEL_GOB' : 'NIVEL_GOBIERNO',
                            'CERTIFICACION' : 'CERTIFICADO',
                            'DEVENGADO' : 'TOTAL_DEVENGADO'}, inplace=True)

hist_gastos = hist_gastos[hist_gastos['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']  
hist_gastos['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
hist_gastos= hist_gastos[hist_gastos['FUENTE'] == '5. RECURSOS DETERMINADOS']
hist_gastos = hist_gastos[hist_gastos['RUBRO'].str[:2] == '18']
hist_gastos= hist_gastos[hist_gastos['ANO_EJE'].notna()]
hist_gastos['CATEGORIA_GTO'].replace({'5. GASTOS CORRIENTES': '5. GASTO CORRIENTE',
                                      '6. GASTOS DE CAPITAL': '6. GASTO DE CAPITAL'}, inplace=True)
hist_gastos1823 = hist_gastos[hist_gastos['ANO_EJE'] != 2024]
hist_gastos1823.info()
hist_gastos1823.head(98942)

<class 'pandas.core.frame.DataFrame'>
Index: 94567 entries, 0 to 99510
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           94567 non-null  int32  
 1   NIVEL_GOBIERNO    94567 non-null  object 
 2   SECTOR            94567 non-null  object 
 3   PLIEGO            94567 non-null  object 
 4   UE                94567 non-null  object 
 5   SEC_EJEC          94567 non-null  object 
 6   FUENTE            94567 non-null  object 
 7   RUBRO             94567 non-null  object 
 8   TIPO_RECURSO      94567 non-null  object 
 9   CATEGORIA_GTO     94567 non-null  object 
 10  TIPO_TRANSACCION  94567 non-null  object 
 11  GENERICA          94567 non-null  object 
 12  SUBGENERICA       94567 non-null  object 
 13  SUBGENERICA_DET   94567 non-null  object 
 14  ESPECIFICA        94567 non-null  object 
 15  ESPECIFICA_DET    94567 non-null  object 
 16  PIA               94567 non-null  int64  
 17

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICADO,COMPROMISO_ANUAL,TOTAL_DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,11. OTROS SERVICIOS,99. SERVICIOS DIVERSOS,0,0,0.0,0.0,211584.99
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,1. DE VEHICULOS,0,0,0.0,0.0,5100.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,"1. SERVICIOS DE CONSULTORIAS, ASESORIAS Y SIMI...",99. OTROS SERVICIOS SIMILARES,0,0,0.0,0.0,8460.00
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,99. COMPRA DE OTROS BIENES,1. COMPRA DE OTROS BIENES,1. HERRAMIENTAS,0,0,0.0,0.0,12773.00
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,11. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. PARA EDIFICIOS Y ESTRUCTURAS,0,0,0.0,0.0,39520.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99506,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,001. GOBIERNO REGIONAL DE LIMA METROPOLITANA,1128,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",99. DE OTROS BIENES Y ACTIVOS,99. DE OTROS BIENES Y ACTIVOS,48046,11246,0.0,0.0,0.00
99507,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,001. GOBIERNO REGIONAL DE LIMA METROPOLITANA,1128,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,3. OTRAS ESTRUCTURAS,2. INFRAESTRUCTURA VIAL,3. COSTO DE CONSTRUCCION POR CONTRATA,0,0,0.0,0.0,406927.14
99508,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,001. GOBIERNO REGIONAL DE LIMA METROPOLITANA,1128,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,8. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,1. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,4. OTROS GASTOS DIVERSOS DE ACTIVOS NO FINANCI...,3. GASTO POR LA CONTRATACION DE SERVICIOS,0,979575,979575.0,0.0,0.00
99509,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,001. GOBIERNO REGIONA

### Informaciòn 2024 (para PMG 2025 -2027)

In [7]:
gastos_tr24 = hist_gastos.copy()
gastos_tr24 = gastos_tr24[gastos_tr24['ANO_EJE'] == 2024]
gastos_tr24= gastos_tr24[gastos_tr24['ANO_EJE'].notna()]
gastos_tr24 = gastos_tr24.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos_tr24.info()
gastos_tr24.head(359)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          215 non-null    int32  
 1   NIVEL_GOBIERNO   215 non-null    object 
 2   SECTOR           215 non-null    object 
 3   PLIEGO           215 non-null    object 
 4   FUENTE           215 non-null    object 
 5   RUBRO            215 non-null    object 
 6   TIPO_RECURSO     215 non-null    object 
 7   CATEGORIA_GTO    215 non-null    object 
 8   GENERICA         215 non-null    object 
 9   TOTAL_DEVENGADO  215 non-null    float64
dtypes: float64(1), int32(1), object(8)
memory usage: 16.1+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,GENERICA,TOTAL_DEVENGADO
0,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0. 0,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,2500.00
1,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0. 0,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,916598.18
2,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. 0,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,134330.00
3,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.00
4,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.00
...,...,...,...,...,...,...,...,...,...,...
210,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. 0,5. GASTO CORRIENTE,5. OTROS GASTOS,270851.70
211,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. 0,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,13252423.12
212,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. 0,7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,8656960.76
213,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.00


In [8]:
#Información historica
gastos_tr_rd_gr = pd.concat([hist_gastos1823, gastos_tr24], ignore_index=True)
gastos_tr_rd_gr.info()
gastos_tr_rd_gr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94782 entries, 0 to 94781
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           94782 non-null  int32  
 1   NIVEL_GOBIERNO    94782 non-null  object 
 2   SECTOR            94782 non-null  object 
 3   PLIEGO            94782 non-null  object 
 4   UE                94567 non-null  object 
 5   SEC_EJEC          94567 non-null  object 
 6   FUENTE            94782 non-null  object 
 7   RUBRO             94782 non-null  object 
 8   TIPO_RECURSO      94782 non-null  object 
 9   CATEGORIA_GTO     94782 non-null  object 
 10  TIPO_TRANSACCION  94567 non-null  object 
 11  GENERICA          94782 non-null  object 
 12  SUBGENERICA       94567 non-null  object 
 13  SUBGENERICA_DET   94567 non-null  object 
 14  ESPECIFICA        94567 non-null  object 
 15  ESPECIFICA_DET    94567 non-null  object 
 16  PIA               94567 non-null  float6

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICADO,COMPROMISO_ANUAL,TOTAL_DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,11. OTROS SERVICIOS,99. SERVICIOS DIVERSOS,0.0,0.0,0.0,0.0,211584.99
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,1. DE VEHICULOS,0.0,0.0,0.0,0.0,5100.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,"1. SERVICIOS DE CONSULTORIAS, ASESORIAS Y SIMI...",99. OTROS SERVICIOS SIMILARES,0.0,0.0,0.0,0.0,8460.00
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,99. COMPRA DE OTROS BIENES,1. COMPRA DE OTROS BIENES,1. HERRAMIENTAS,0.0,0.0,0.0,0.0,12773.00
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,11. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. PARA EDIFICIOS Y ESTRUCTURAS,0.0,0.0,0.0,0.0,39520.50


### Cálculo de ratio por GG y Tipo de Recurso

In [9]:
ratios_preliminar = gastos_tr_rd_gr.copy()
ratios_preliminar = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
ratios_preliminar['CL_TIPO_RECURSO'] = ' '
ratios_preliminar['CL_TIPO_RECURSO'] = np.where((ratios_preliminar['TIPO_RECURSO'].str.contains('CANON'))|(ratios_preliminar['TIPO_RECURSO'].str.contains('SOBRECANON'))|
                                                (ratios_preliminar['TIPO_RECURSO'].str.contains('REGALIA'))|(ratios_preliminar['TIPO_RECURSO'].str.contains('MINERA')), 'C_SC_RM',
                                                np.where(ratios_preliminar['TIPO_RECURSO'].str.contains('ADUANA'), 'R_Aduana',
                                                         np.where(ratios_preliminar['TIPO_RECURSO'].str.contains('FED'), 'FED',
                                                                  np.where(ratios_preliminar['TIPO_RECURSO'].str.contains('FIDEICOMISO'), 'FIDEICOMISO',
                                                                           np.where(ratios_preliminar['CL_TIPO_RECURSO'] == ' ', 'OTROS', ratios_preliminar['CL_TIPO_RECURSO'])))))                                                 

ratios_preliminar = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
ratios_preliminar['DEN_DEVENGADO'] = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['TOTAL_DEVENGADO'].transform('sum').astype(float)

#Generar ratio_
ratios_preliminar['ratio_'] = ratios_preliminar['TOTAL_DEVENGADO'] / ratios_preliminar['DEN_DEVENGADO']

# Filtrando los datos para mantener solo los años entre 2018 y 2024
ratios_preliminar = ratios_preliminar[(ratios_preliminar['ANO_EJE'] >= 2018) & (ratios_preliminar['ANO_EJE'] <= 2024)]

# Eliminando filas donde ratio_ sea nulo

ratios_preliminar = ratios_preliminar.dropna(subset=['ratio_'])
# Definir las estadísticas a calcular
estadisticas = ['min', 'mean', 'max']

# Iterar sobre cada estadística
for estadistica in estadisticas:
    # Calcular la estadística 'estadistica' del ratio_ por grupos y agregarla al dataframe
    ratios_preliminar[f'{estadistica}_ratio_'] = ratios_preliminar.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'CL_TIPO_RECURSO'])['ratio_'].transform(estadistica)

# Ordenar el dataframe por las columnas especificadas
ratios_preliminar.sort_values(by=['PLIEGO', 'GENERICA', 'CL_TIPO_RECURSO'], inplace=True)

# Eliminar filas duplicadas en el dataframe
ratios_preliminar.drop_duplicates(inplace=True)

# Reshape wide
ratios_preliminar = ratios_preliminar.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'], columns='CL_TIPO_RECURSO', values=['min_ratio_', 'mean_ratio_', 'max_ratio_']).reset_index().rename_axis(None)
ratios_preliminar.columns = [f'{i}{j}' for i, j in ratios_preliminar.columns]
ratios_preliminar.info()
ratios_preliminar.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NIVEL_GOBIERNO          128 non-null    object 
 1   SECTOR                  128 non-null    object 
 2   PLIEGO                  128 non-null    object 
 3   FUENTE                  128 non-null    object 
 4   RUBRO                   128 non-null    object 
 5   CATEGORIA_GTO           128 non-null    object 
 6   GENERICA                128 non-null    object 
 7   max_ratio_C_SC_RM       103 non-null    float64
 8   max_ratio_FED           54 non-null     float64
 9   max_ratio_FIDEICOMISO   6 non-null      float64
 10  max_ratio_OTROS         123 non-null    float64
 11  max_ratio_R_Aduana      8 non-null      float64
 12  mean_ratio_C_SC_RM      103 non-null    float64
 13  mean_ratio_FED          54 non-null     float64
 14  mean_ratio_FIDEICOMISO  6 non-null      fl

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,max_ratio_C_SC_RM,max_ratio_FED,max_ratio_FIDEICOMISO,...,mean_ratio_C_SC_RM,mean_ratio_FED,mean_ratio_FIDEICOMISO,mean_ratio_OTROS,mean_ratio_R_Aduana,min_ratio_C_SC_RM,min_ratio_FED,min_ratio_FIDEICOMISO,min_ratio_OTROS,min_ratio_R_Aduana
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000000,0.993398,NaN,...,0.000000,0.816308,NaN,0.358127,NaN,0.000000,0.583897,NaN,0.0,NaN
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,NaN
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.416103,NaN,...,1.000000,0.183692,NaN,0.641873,NaN,1.000000,0.006602,NaN,0.0,NaN
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,2. PENSIONES Y OTRAS PRESTACIONES SOCIALES,0.012342,NaN,NaN,...,0.006184,NaN,NaN,0.000000,NaN,0.000026,NaN,NaN,0.0,NaN
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.259257,0.846079,NaN,...,0.105649,0.752500,NaN,0.529666,NaN,0.032345,0.646696,NaN,0.0,NaN


In [10]:
gastos = gastos_tr_rd_gr.copy()
gastos = gastos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos['CL_TIPO_RECURSO'] = ' '
gastos['CL_TIPO_RECURSO'] = np.where((gastos['TIPO_RECURSO'].str.contains('CANON'))|(gastos['TIPO_RECURSO'].str.contains('SOBRECANON'))|
                                       (gastos['TIPO_RECURSO'].str.contains('REGALIA'))|(gastos['TIPO_RECURSO'].str.contains('MINERA')), 'C_SC_RM',
                                            np.where(gastos['TIPO_RECURSO'].str.contains('ADUANA'), 'R_Aduana',
                                                              np.where(gastos['TIPO_RECURSO'].str.contains('FIDEICOMISO'), 'FIDEICOMISO',
                                                                       np.where(gastos['CL_TIPO_RECURSO'] == ' ', 'OTROS', gastos['CL_TIPO_RECURSO'])))) 

gastos = gastos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos.info()
gastos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          352 non-null    int32  
 1   NIVEL_GOBIERNO   352 non-null    object 
 2   SECTOR           352 non-null    object 
 3   PLIEGO           352 non-null    object 
 4   FUENTE           352 non-null    object 
 5   RUBRO            352 non-null    object 
 6   CL_TIPO_RECURSO  352 non-null    object 
 7   TOTAL_DEVENGADO  352 non-null    float64
dtypes: float64(1), int32(1), object(6)
memory usage: 20.8+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CL_TIPO_RECURSO,TOTAL_DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,0.000000e+00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",OTROS,6.296947e+06
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,3.353061e+08
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",OTROS,6.127386e+06
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,5.188509e+07


## Informaciòn Ingresos

### Conversión de BD (a nivel de tipo de recurso) 2018 - 2023

In [11]:
bd_ingresos_tr.info()
bd_ingresos_tr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25366 entries, 0 to 25365
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           25366 non-null  int32  
 1   NIVEL_GOB         25366 non-null  object 
 2   SECTOR            25366 non-null  object 
 3   PLIEGO            25366 non-null  object 
 4   EJECUTORA         25366 non-null  object 
 5   SEC_EJEC          25366 non-null  object 
 6   FUENTE            25366 non-null  object 
 7   RUBRO             25366 non-null  object 
 8   TIPO_RECURSO      25366 non-null  object 
 9   TIPO_TRANSACCION  25366 non-null  object 
 10  GENERICA          25366 non-null  object 
 11  SUBGENERICA       25366 non-null  object 
 12  SUBGENERICA_DET   25366 non-null  object 
 13  ESPECIFICA        25366 non-null  object 
 14  ESPECIFICA_DET    25366 non-null  object 
 15  PIA               25366 non-null  int64  
 16  PIM               25366 non-null  int64 

,ANO_EJE,NIVEL_GOB,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,2. DONACIONES DE CAPITAL,3. DE OTRAS UNIDADES DE GOBIERNO,1. DE OTRAS UNIDADES DE GOBIERNO,5. DE FONDOS PUBLICOS,0,36400,0.00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,94377.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",D. SUB CUENTA - PARTICIPACIONES - DNTP,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,2484.26
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,1546.60
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,69121.51


In [12]:
hist_ingresos = bd_ingresos_tr.copy()
# Reemplazar comas por puntos en la columna 'recaudacion'
hist_ingresos.rename(columns={'RECAUDADO': 'RECAUDACION',
                              'NIVEL_GOB': 'NIVEL_GOBIERNO'}, inplace=True)

# Convertir la columna 'recaudacion' a tipo numérico si es posible
hist_ingresos['RECAUDACION'] = hist_ingresos['RECAUDACION'].astype(float)

# Seleccionar solo GOBIERNOS REGIONALES
hist_ingresos= hist_ingresos[hist_ingresos['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
# Seleccionar solo RUBRO 18 - CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES.
hist_ingresos= hist_ingresos[hist_ingresos['FUENTE'] == '5. RECURSOS DETERMINADOS']
hist_ingresos['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"

hist_ingresos = hist_ingresos[hist_ingresos['RUBRO'].str[:2] == '18']

# Dropear el año 2024, porque esa informaciòn la brinda DPSP
hist_ingresos1823 = hist_ingresos[hist_ingresos['ANO_EJE'] != 2024]

hist_ingresos1823.info()
hist_ingresos1823.head()

<class 'pandas.core.frame.DataFrame'>
Index: 21619 entries, 0 to 22403
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           21619 non-null  int32  
 1   NIVEL_GOBIERNO    21619 non-null  object 
 2   SECTOR            21619 non-null  object 
 3   PLIEGO            21619 non-null  object 
 4   EJECUTORA         21619 non-null  object 
 5   SEC_EJEC          21619 non-null  object 
 6   FUENTE            21619 non-null  object 
 7   RUBRO             21619 non-null  object 
 8   TIPO_RECURSO      21619 non-null  object 
 9   TIPO_TRANSACCION  21619 non-null  object 
 10  GENERICA          21619 non-null  object 
 11  SUBGENERICA       21619 non-null  object 
 12  SUBGENERICA_DET   21619 non-null  object 
 13  ESPECIFICA        21619 non-null  object 
 14  ESPECIFICA_DET    21619 non-null  object 
 15  PIA               21619 non-null  int64  
 16  PIM               21619 non-null  int64  
 17

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDACION
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,2. DONACIONES DE CAPITAL,3. DE OTRAS UNIDADES DE GOBIERNO,1. DE OTRAS UNIDADES DE GOBIERNO,5. DE FONDOS PUBLICOS,0,36400,0.00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,94377.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",D. SUB CUENTA - PARTICIPACIONES - DNTP,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,2484.26
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,1546.60
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,69121.51


### Informaciòn 2024 (para PMG 2025-2027)

In [13]:
ingresos_tr24 = hist_ingresos.copy()
ingresos_tr24 = ingresos_tr24[ingresos_tr24['ANO_EJE'] == 2024]
ingresos_tr24 = ingresos_tr24.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()
ingresos_tr24.info()
ingresos_tr24.head(337)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ANO_EJE         323 non-null    int32  
 1   NIVEL_GOBIERNO  323 non-null    object 
 2   SECTOR          323 non-null    object 
 3   PLIEGO          323 non-null    object 
 4   FUENTE          323 non-null    object 
 5   RUBRO           323 non-null    object 
 6   TIPO_RECURSO    323 non-null    object 
 7   RECAUDACION     323 non-null    float64
dtypes: float64(1), int32(1), object(6)
memory usage: 19.1+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_RECURSO,RECAUDACION
0,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",10. SUB CUENTA - PARTICIPACIONES - BOI,39703.58
1,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,1661367.77
2,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",26. SUB CUENTA - PARTICIPACIONES - FIDT,46000.00
3,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",27. SUB CUENTA - FONCOR - LEY 31069,312781.99
4,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,0.00
...,...,...,...,...,...,...,...,...
318,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",I. SUB CUENTA - CANON HIDROENERGETICO,2757766.17
319,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,2158.72
320,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...","O. SUB CUENTA - CANON PESQUERO, DERECHOS",40351.07
321,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",P. SUB CUENTA - REGALIAS MINERAS,2897414.97


In [14]:
#Información historica
ingresos_tr_rd_gr = pd.concat([hist_ingresos1823, ingresos_tr24], ignore_index=True)
ingresos_tr_rd_gr.info()
ingresos_tr_rd_gr.head(21942)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21942 entries, 0 to 21941
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           21942 non-null  int32  
 1   NIVEL_GOBIERNO    21942 non-null  object 
 2   SECTOR            21942 non-null  object 
 3   PLIEGO            21942 non-null  object 
 4   EJECUTORA         21619 non-null  object 
 5   SEC_EJEC          21619 non-null  object 
 6   FUENTE            21942 non-null  object 
 7   RUBRO             21942 non-null  object 
 8   TIPO_RECURSO      21942 non-null  object 
 9   TIPO_TRANSACCION  21619 non-null  object 
 10  GENERICA          21619 non-null  object 
 11  SUBGENERICA       21619 non-null  object 
 12  SUBGENERICA_DET   21619 non-null  object 
 13  ESPECIFICA        21619 non-null  object 
 14  ESPECIFICA_DET    21619 non-null  object 
 15  PIA               21619 non-null  float64
 16  PIM               21619 non-null  float6

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDACION
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,2. DONACIONES DE CAPITAL,3. DE OTRAS UNIDADES DE GOBIERNO,1. DE OTRAS UNIDADES DE GOBIERNO,5. DE FONDOS PUBLICOS,0.0,36400.0,0.00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0.0,0.0,94377.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",D. SUB CUENTA - PARTICIPACIONES - DNTP,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,2484.26
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,1546.60
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0.0,0.0,69121.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21937,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",I. SUB CUENTA - CANON HIDROENERGETICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2757766.17
21938,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2158.72
21939,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...","O. SUB CUENTA - CANON PESQUERO, DERECHOS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40351.07
21940,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",P. SUB CUENTA - REGALIAS MINERAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2897414.97


### Cálculo de saldo de balance por tipo de recurso 

In [15]:
ingresos = ingresos_tr_rd_gr.copy()
ingresos = ingresos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()
ingresos['CL_TIPO_RECURSO'] = ' '
ingresos['CL_TIPO_RECURSO'] = np.where((ingresos['TIPO_RECURSO'].str.contains('CANON'))|(ingresos['TIPO_RECURSO'].str.contains('SOBRECANON'))|
                                       (ingresos['TIPO_RECURSO'].str.contains('REGALIA'))|(ingresos['TIPO_RECURSO'].str.contains('MINERA')), 'C_SC_RM',
                                            np.where(ingresos['TIPO_RECURSO'].str.contains('ADUANA'), 'R_Aduana',
                                                     np.where(ingresos['TIPO_RECURSO'].str.contains('FED'), 'FED',
                                                              np.where(ingresos['TIPO_RECURSO'].str.contains('FIDEICOMISO'), 'FIDEICOMISO',
                                                                       np.where(ingresos['CL_TIPO_RECURSO'] == ' ', 'OTROS', ingresos['CL_TIPO_RECURSO']))))) 
ingresos = ingresos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()

ingresos.info()
ingresos.head(597)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          687 non-null    int32  
 1   NIVEL_GOBIERNO   687 non-null    object 
 2   SECTOR           687 non-null    object 
 3   PLIEGO           687 non-null    object 
 4   FUENTE           687 non-null    object 
 5   RUBRO            687 non-null    object 
 6   CL_TIPO_RECURSO  687 non-null    object 
 7   RECAUDACION      687 non-null    float64
dtypes: float64(1), int32(1), object(6)
memory usage: 40.4+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CL_TIPO_RECURSO,RECAUDACION
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,7.101308e+05
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",FED,9.067271e+06
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",FIDEICOMISO,7.066007e+04
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",OTROS,1.976071e+05
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,5.416683e+08
...,...,...,...,...,...,...,...,...
592,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",FIDEICOMISO,5.941928e+04
593,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",OTROS,6.066520e+05
594,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",C_SC_RM,3.670301e+08
595,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",FED,2.147122e+06


In [16]:
sb_ratios = pd.merge(gastos, ingresos, on=['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'], how='outer', validate = '1:1')
postmerge(sb_ratios)
sb_ratios = sb_ratios[(sb_ratios['ANO_EJE'] >= 2018) & (sb_ratios['ANO_EJE'] <= 2024)]
# Eliminar las filas donde 'recaudacion' sea igual a NaN y 'total_devengado' sea igual a 0
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'].isna()) & (sb_ratios['TOTAL_DEVENGADO'] == 0)].index)
# Eliminar las filas donde 'recaudacion' sea igual a 0 y 'total_devengado' sea igual a 0
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'] == 0) & (sb_ratios['TOTAL_DEVENGADO'] == 0)].index)
# Eliminar las filas donde 'recaudacion' sea igual a 0 y 'total_devengado' sea igual a NaN
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'] == 0) & (sb_ratios['TOTAL_DEVENGADO'].isna())].index)
# Eliminar las filas donde 'recaudacion' sea igual a NaN y 'total_devengado' sea igual a NaN
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'].isna()) & (sb_ratios['TOTAL_DEVENGADO'].isna())].index)

# Reemplazar los valores nulos en 'total_devengado' y 'recaudacion' con 0
sb_ratios['TOTAL_DEVENGADO'].fillna(0, inplace=True)
sb_ratios['RECAUDACION'].fillna(0, inplace=True)

# Calcular la variable 'sb' y reemplazar los valores negativos con 0
sb_ratios['sb'] = sb_ratios['RECAUDACION'] - sb_ratios['TOTAL_DEVENGADO']
sb_ratios['sb'] = np.where((sb_ratios['sb'] < 0), 0, sb_ratios['sb'])

# Calcular la suma acumulada de 'sb' por grupos y guardarla en 'den_sb'
sb_ratios['den_sb'] = sb_ratios.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['sb'].transform('sum')
sb_ratios['ratio_sb'] = sb_ratios['sb'] / sb_ratios['den_sb']

# Definir las estadísticas a calcular
estadisticas = ['min', 'mean', 'max']

# Iterar sobre cada estadística
for estadistica in estadisticas:
    # Calcular la estadística 'estadistica' del ratio_ por grupos y agregarla al dataframe
    sb_ratios[f'{estadistica}_ratio_sb_'] = sb_ratios.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['ratio_sb'].transform(estadistica)

# Eliminar filas duplicadas en el dataframe
sb_ratios.drop_duplicates(inplace=True)

# Reshape wide
sb_ratios = sb_ratios.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='CL_TIPO_RECURSO', values=['min_ratio_sb_', 'mean_ratio_sb_', 'max_ratio_sb_']).reset_index().rename_axis(None)
sb_ratios.columns = [f'{i}{j}' for i, j in sb_ratios.columns]

sb_ratios.info()
sb_ratios.head(27)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   NIVEL_GOBIERNO             26 non-null     object 
 1   SECTOR                     26 non-null     object 
 2   PLIEGO                     26 non-null     object 
 3   FUENTE                     26 non-null     object 
 4   RUBRO                      26 non-null     object 
 5   max_ratio_sb_C_SC_RM       26 non-null     float64
 6   max_ratio_sb_FED           25 non-null     float64
 7   max_ratio_sb_FIDEICOMISO   23 non-null     float64
 8   max_ratio_sb_OTROS         26 non-null     float64
 9   max_ratio_sb_R_Aduana      2 non-null      float64
 10  mean_ratio_sb_C_SC_RM      26 non-null     float64
 11  mean_ratio_sb_FED          25 non-null     float64
 12  mean_ratio_sb_FIDEICOMISO  23 non-null     float64
 13  mean_ratio_sb_OTROS        26 non-null     float64
 

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,max_ratio_sb_C_SC_RM,max_ratio_sb_FED,max_ratio_sb_FIDEICOMISO,max_ratio_sb_OTROS,max_ratio_sb_R_Aduana,mean_ratio_sb_C_SC_RM,mean_ratio_sb_FED,mean_ratio_sb_FIDEICOMISO,mean_ratio_sb_OTROS,mean_ratio_sb_R_Aduana,min_ratio_sb_C_SC_RM,min_ratio_sb_FED,min_ratio_sb_FIDEICOMISO,min_ratio_sb_OTROS,min_ratio_sb_R_Aduana
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.721276,0.970558,0.030819,0.378706,NaN,0.357733,0.570227,0.008830,0.063210,NaN,0.021239,0.212496,2.464554e-03,0.000000,NaN
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.990814,0.046389,0.004934,0.181754,NaN,0.949505,0.022306,0.003892,0.025965,NaN,0.804106,0.005796,2.523245e-03,0.000000,NaN
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.866372,0.475479,0.038002,0.280500,NaN,0.568809,0.295271,0.022230,0.113690,NaN,0.325419,0.107003,1.113492e-02,0.000000,NaN
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.983306,0.179164,0.000166,0.668018,8.458973e-09,0.835782,0.065613,0.000086,0.098543,7.365513e-09,0.316011,0.015962,9.173420e-06,0.000000,6.272054e-09
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.734951,0.638826,0.000489,0.763075,NaN,0.404551,0.377726,0.000250,0.217473,NaN,0.181421,0.055456,4.794129e-05,0.000000,NaN
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.959700,0.143736,NaN,0.735446,NaN,0.776380,0.069173,NaN,0.154447,NaN,0.248247,0.016306,NaN,0.000000,NaN
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.996337,0.123578,0.000551,0.000000,NaN,0.955268,0.044568,0.000164,0.000000,NaN,0.875871,0.003628,3.436889e-05,0.000000,NaN
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.380141,0.272659,0.000627,0.754074,NaN,0.210489,0.129775,0.000433,0.659550,NaN,0.067908,0.042281,1.417107e-04,0.483894,NaN
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.251217,0.860014,0.157718,0.963693,NaN,0.106029,0.601542,0.093395,0.199034,NaN,0.001829,0.029884,4.593988e-03,0.000000,NaN
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.963473,0.214245,0.000003,0.614428,NaN,0.570149,0.063413,0.000001,0.366437,NaN,0.274407,0.008792,1.411947e-09,0.000000,NaN


In [17]:
aduanas_callao_i= ingresos_tr_rd_gr.copy()
# Seleccionar solo si TIPO DE RECURSO es ADUANA
aduanas_callao_i = aduanas_callao_i[aduanas_callao_i['TIPO_RECURSO'].str.contains('ADUANA')] 
# Seleccionar solo si el pliego 464.GOBIERNO REGIONAL DEL CALLAO
aduanas_callao_i = aduanas_callao_i[aduanas_callao_i['PLIEGO'].str[:3] == '464'].reset_index(drop = True)
aduanas_callao_i.info()
aduanas_callao_i.head(105)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           164 non-null    int32  
 1   NIVEL_GOBIERNO    164 non-null    object 
 2   SECTOR            164 non-null    object 
 3   PLIEGO            164 non-null    object 
 4   EJECUTORA         162 non-null    object 
 5   SEC_EJEC          162 non-null    object 
 6   FUENTE            164 non-null    object 
 7   RUBRO             164 non-null    object 
 8   TIPO_RECURSO      164 non-null    object 
 9   TIPO_TRANSACCION  162 non-null    object 
 10  GENERICA          162 non-null    object 
 11  SUBGENERICA       162 non-null    object 
 12  SUBGENERICA_DET   162 non-null    object 
 13  ESPECIFICA        162 non-null    object 
 14  ESPECIFICA_DET    162 non-null    object 
 15  PIA               162 non-null    float64
 16  PIM               162 non-null    float64
 1

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDACION
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0.0,0.0,1.404864e+07
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",B. RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0.0,0.0,8.626312e+04
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,3.308862e+05
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,3. PARTICIPACION RENTA DE ADUANAS,1. RENTA DE ADUANAS,0.0,0.0,2.006127e+08
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",B. RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,3.061600e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,3. PARTICIPACION RENTA DE ADUANAS,1. RENTA DE ADUANAS,0.0,0.0,1.495608e+06
101,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,2.005360e+03
102,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0.0,0.0,1.431160e+04
103,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,404. HOSPITAL DE REHABILITACIÓN DEL CALLAO,1667,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,1. I

In [18]:
aduanas_callao_g= gastos_tr_rd_gr.copy()
# Seleccionar solo si TIPO DE RECURSO es ADUANA
aduanas_callao_g = aduanas_callao_g[aduanas_callao_g['TIPO_RECURSO'].str.contains('ADUANA')] 
# Seleccionar solo si el pliego 464.GOBIERNO REGIONAL DEL CALLAO
aduanas_callao_g = aduanas_callao_g[aduanas_callao_g['PLIEGO'].str[:3] == '464'].reset_index(drop = True)
aduanas_callao_g.info()
aduanas_callao_g.head(2726)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           1564 non-null   int32  
 1   NIVEL_GOBIERNO    1564 non-null   object 
 2   SECTOR            1564 non-null   object 
 3   PLIEGO            1564 non-null   object 
 4   UE                1564 non-null   object 
 5   SEC_EJEC          1564 non-null   object 
 6   FUENTE            1564 non-null   object 
 7   RUBRO             1564 non-null   object 
 8   TIPO_RECURSO      1564 non-null   object 
 9   CATEGORIA_GTO     1564 non-null   object 
 10  TIPO_TRANSACCION  1564 non-null   object 
 11  GENERICA          1564 non-null   object 
 12  SUBGENERICA       1564 non-null   object 
 13  SUBGENERICA_DET   1564 non-null   object 
 14  ESPECIFICA        1564 non-null   object 
 15  ESPECIFICA_DET    1564 non-null   object 
 16  PIA               1564 non-null   float64


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICADO,COMPROMISO_ANUAL,TOTAL_DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,3. SERVICIO DE CAPACITACION Y PERFECCIONAMIENTO,2. REALIZADO POR PERSONAS NATURALES,0.0,0.0,0.0,0.0,32500.00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,5. GASTO CORRIENTE,...,1. PERSONAL Y OBLIGACIONES SOCIALES,3. CONTRIBUCIONES A LA SEGURIDAD SOCIAL,1. OBLIGACIONES DEL EMPLEADOR,1. OBLIGACIONES DEL EMPLEADOR,5. CONTRIBUCIONES A ESSALUD,0.0,0.0,0.0,0.0,1273381.07
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,8. CONTRATO ADMINISTRATIVO DE SERVICIOS,1. CONTRATO ADMINISTRATIVO DE SERVICIOS,1. CONTRATO ADMINISTRATIVO DE SERVICIOS,0.0,0.0,0.0,0.0,6663638.82
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...",9. ADQUISICION DE MAQUINARIA Y EQUIPO DIVERSOS,"2. ASEO, LIMPIEZA Y COCINA",0.0,0.0,0.0,0.0,32767.31
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,99. OTROS ACCESORIOS Y REPUESTOS,0.0,0.0,0.0,0.0,82330.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...","4. ADQUISICION DE MOBILIARIO, EQUIPOS Y APARAT...",1. MOBILIARIO,0.0,0.0,0.0,0.0,20865.76
1560,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,5. GASTO CORRIENTE,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,8. CONTRATO ADMINISTRATIVO DE SERVICIOS,1. CONTRATO ADMINISTRATIVO DE SERVICIOS,2. CONTRIBUCIONES A ESSALUD DE C.A.S.,0.0,0.0,0.0,0.0,80837.91
1561,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,403. HOSPITAL DE VENTANILLA,1452,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",Q. SUB CUENTA - RENTAS DE ADUANA,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS"

# 2. Insumos PMG 2025-2027

## Anàlisis a nivel de gastos: ratio por generica de gasto y tipo de recurso

In [19]:
ratios_preliminar.info()
ratios_preliminar.head(139)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NIVEL_GOBIERNO          128 non-null    object 
 1   SECTOR                  128 non-null    object 
 2   PLIEGO                  128 non-null    object 
 3   FUENTE                  128 non-null    object 
 4   RUBRO                   128 non-null    object 
 5   CATEGORIA_GTO           128 non-null    object 
 6   GENERICA                128 non-null    object 
 7   max_ratio_C_SC_RM       103 non-null    float64
 8   max_ratio_FED           54 non-null     float64
 9   max_ratio_FIDEICOMISO   6 non-null      float64
 10  max_ratio_OTROS         123 non-null    float64
 11  max_ratio_R_Aduana      8 non-null      float64
 12  mean_ratio_C_SC_RM      103 non-null    float64
 13  mean_ratio_FED          54 non-null     float64
 14  mean_ratio_FIDEICOMISO  6 non-null      fl

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,max_ratio_C_SC_RM,max_ratio_FED,max_ratio_FIDEICOMISO,...,mean_ratio_C_SC_RM,mean_ratio_FED,mean_ratio_FIDEICOMISO,mean_ratio_OTROS,mean_ratio_R_Aduana,min_ratio_C_SC_RM,min_ratio_FED,min_ratio_FIDEICOMISO,min_ratio_OTROS,min_ratio_R_Aduana
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000000,0.993398,NaN,...,0.000000,0.816308,NaN,0.358127,NaN,0.000000,0.583897,NaN,0.000000,NaN
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.416103,NaN,...,1.000000,0.183692,NaN,0.641873,NaN,1.000000,0.006602,NaN,0.000000,NaN
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,2. PENSIONES Y OTRAS PRESTACIONES SOCIALES,0.012342,NaN,NaN,...,0.006184,NaN,NaN,0.000000,NaN,0.000026,NaN,NaN,0.000000,NaN
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.259257,0.846079,NaN,...,0.105649,0.752500,NaN,0.529666,NaN,0.032345,0.646696,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.310613,NaN,...,1.000000,0.168131,NaN,0.213246,0.215707,1.000000,0.053685,NaN,0.213246,0.048554
124,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1.000000,NaN,NaN,...,1.000000,NaN,NaN,0.139300,0.120048,1.000000,NaN,NaN,0.139300,0.070637
125,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.013387,NaN,NaN,...,0.006006,NaN,NaN,1.000000,NaN,0.000000,NaN,NaN,1.000000,NaN
126,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.162404,NaN,NaN,...,0.162404,NaN,NaN,NaN,NaN,0.162404,NaN,NaN,NaN,NaN


In [20]:
ratios_preliminar_red = ratios_preliminar.copy()
ratios_preliminar_red = ratios_preliminar_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'mean_ratio_C_SC_RM', 'mean_ratio_FED', 'mean_ratio_FIDEICOMISO',  'mean_ratio_OTROS', 'mean_ratio_R_Aduana']]
# Calcular el ratio tot_rat
ratios_preliminar_red['tot_rat'] = ratios_preliminar_red[['mean_ratio_C_SC_RM', 'mean_ratio_FED', 'mean_ratio_FIDEICOMISO',  'mean_ratio_OTROS', 'mean_ratio_R_Aduana']].sum(axis=1)

#eliminar si tot_rat es =0 o . :
ratios_preliminar_red['tot_rat'] = round(ratios_preliminar_red['tot_rat'], 8)
ratios_preliminar_red = ratios_preliminar_red[(ratios_preliminar_red['tot_rat'].fillna(0) != 0)]

#Asumimos que ningún pliego debe financiar personal con RD a excepcion de GR Callao
ratios_preliminar_red = ratios_preliminar_red[~(((ratios_preliminar_red['GENERICA'].str[0] == '1') | (ratios_preliminar_red['GENERICA'].str[0] == '2')) & (ratios_preliminar_red['PLIEGO'].str[:3] != '464'))]
#Estamos cubriendo GG 2.8 en función al monto de Tesoro y no al histórico
ratios_preliminar_red = ratios_preliminar_red[(ratios_preliminar_red['GENERICA'].str[0] != '8')]
#Mantener las siguientes variables
ratios_preliminar_red = ratios_preliminar_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'mean_ratio_C_SC_RM', 'mean_ratio_R_Aduana']]

#CANON Y SOBRECANON
#Corregir la participación de acuerdo a la participación de cada Genérica
# Calcular la suma acumulada de mean_ratio_C_SC_RM para cada grupo
ratios_preliminar_red['Tot1'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['mean_ratio_C_SC_RM'].transform('sum')

# Calcular el ratio_f_c
ratios_preliminar_red['ratio_f_c'] = ratios_preliminar_red['mean_ratio_C_SC_RM'] / ratios_preliminar_red['Tot1']

# Corregir ratio_f_c si ratio_f es mayor que 0.2 y la primera letra de generica es "3"
ratios_preliminar_red['ratio_f_c'] = np.where((ratios_preliminar_red['ratio_f_c'] > 0.2) & (ratios_preliminar_red['GENERICA'].str[0] == '3'), 0.2, ratios_preliminar_red['ratio_f_c'])

# Calcular la suma acumulada de ratio_f_c para cada grupo
ratios_preliminar_red['Tot2'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['ratio_f_c'].transform('sum')

# Calcular comp
ratios_preliminar_red['comp'] = 1 - ratios_preliminar_red['Tot2']

# Ajustar ratio_f_c si la primera letra de generica es "6"
ratios_preliminar_red['ratio_f_c'] = ratios_preliminar_red['ratio_f_c'].fillna(0)
ratios_preliminar_red['comp'] = ratios_preliminar_red['comp'].fillna(0)
ratios_preliminar_red['ratio_f_c'] = np.where(ratios_preliminar_red['GENERICA'].str[0] == '6', ratios_preliminar_red[['ratio_f_c', 'comp']].sum(axis=1), ratios_preliminar_red['ratio_f_c'])

# Eliminar columnas temporales
ratios_preliminar_red.drop(columns=['Tot1', 'Tot2'], inplace=True)

#RENTA DE ADUANA
#Corregir la participación de acuerdo a la participación de cada Genérica
# Calcular la suma acumulada de Tot_ra para cada grupo
ratios_preliminar_red['Tot_ra'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['mean_ratio_R_Aduana'].transform('sum')
# Calcular ratioratio_f_ra
ratios_preliminar_red['ratio_f_ra'] = ratios_preliminar_red['mean_ratio_R_Aduana'] / ratios_preliminar_red['Tot_ra']

# Calcular la suma acumulada de ratio_f_ra para cada grupo
ratios_preliminar_red['Totra2'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['ratio_f_ra'].transform('sum')
ratios_preliminar_red['compra'] = 1 - ratios_preliminar_red['Totra2']

# Ajustar ratio_f_ra si la primera letra de generica es "6"
ratios_preliminar_red['ratio_f_ra']= ratios_preliminar_red['ratio_f_ra'].fillna(0)
ratios_preliminar_red['compra']= ratios_preliminar_red['compra'].fillna(0)
ratios_preliminar_red['ratio_f_ra'] = np.where(ratios_preliminar_red['GENERICA'].str[0] == '6', ratios_preliminar_red[['ratio_f_ra', 'compra']].sum(axis=1), ratios_preliminar_red['ratio_f_ra'])

# Eliminar columnas temporales
ratios_preliminar_red.drop(columns=['Tot_ra', 'Totra2'], inplace=True)

# Eliminar filas duplicadas en el dataframe
sb_ratios.drop_duplicates(inplace=True)


ratios_preliminar_red.info()
ratios_preliminar_red.head(139)

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 127
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NIVEL_GOBIERNO       98 non-null     object 
 1   SECTOR               98 non-null     object 
 2   PLIEGO               98 non-null     object 
 3   FUENTE               98 non-null     object 
 4   RUBRO                98 non-null     object 
 5   CATEGORIA_GTO        98 non-null     object 
 6   GENERICA             98 non-null     object 
 7   mean_ratio_C_SC_RM   81 non-null     float64
 8   mean_ratio_R_Aduana  7 non-null      float64
 9   ratio_f_c            98 non-null     float64
 10  comp                 98 non-null     float64
 11  ratio_f_ra           98 non-null     float64
 12  compra               98 non-null     float64
dtypes: float64(6), object(7)
memory usage: 10.7+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,mean_ratio_C_SC_RM,mean_ratio_R_Aduana,ratio_f_c,comp,ratio_f_ra,compra
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000000,NaN,0.000000,0.0,0.000000,1.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,NaN,1.000000,0.0,1.000000,1.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.105649,NaN,0.093246,0.0,0.000000,1.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,5. OTROS GASTOS,NaN,NaN,0.000000,0.0,0.000000,1.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.336875,NaN,0.297327,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,1.000000,0.102093,0.400000,0.0,0.109604,0.0
123,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.215707,0.400000,0.0,0.231577,0.0
125,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.006006,NaN,0.005281,0.0,0.000000,1.0
126,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,465. MUNICIPALIDAD METROPOLITANA DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.162404,NaN,0.142793,0.0,0.000000,1.0


In [23]:
outputFile = os.path.join(r'Y:\1) NUEVO compartido DAPT\Financiamiento_Data\PMG 2025-2027\Outputs','ratios_preliminar_red'+ fecha_corte+ ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    ratios_preliminar_red.to_excel(ew, sheet_name="Ratios", index = False)
#%%

## Adaptar a los ratios que nos interesan (Tipo de recurso de ingresos- SB)

In [25]:
sb_ratios_red = sb_ratios.copy()
sb_ratios_red = sb_ratios_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'mean_ratio_sb_C_SC_RM', 'mean_ratio_sb_FED', 'mean_ratio_sb_FIDEICOMISO', 'mean_ratio_sb_OTROS', 'mean_ratio_sb_R_Aduana']]
# Calcular el ratio tot_rat
sb_ratios_red['tot_rat'] = sb_ratios_red[['mean_ratio_sb_C_SC_RM', 'mean_ratio_sb_FED', 'mean_ratio_sb_FIDEICOMISO', 'mean_ratio_sb_OTROS', 'mean_ratio_sb_R_Aduana']].sum(axis=1)

# Calcular los ratios
sb_ratios_red['msb_c'] = sb_ratios_red['mean_ratio_sb_C_SC_RM'] / sb_ratios_red['tot_rat']
sb_ratios_red['ratio_sb_fed'] = sb_ratios_red['mean_ratio_sb_FED'] / sb_ratios_red['tot_rat']
sb_ratios_red['msb_fide'] = sb_ratios_red['mean_ratio_sb_FIDEICOMISO'] / sb_ratios_red['tot_rat']
sb_ratios_red['msb_ot'] = sb_ratios_red['mean_ratio_sb_OTROS'] / sb_ratios_red['tot_rat']
sb_ratios_red['ratio_sb_ra'] = sb_ratios_red['mean_ratio_sb_R_Aduana'] / sb_ratios_red['tot_rat']

# Eliminar columnas temporales
sb_ratios_red.drop(columns=['tot_rat', 'mean_ratio_sb_C_SC_RM', 'mean_ratio_sb_FED', 'mean_ratio_sb_FIDEICOMISO', 'mean_ratio_sb_OTROS', 'mean_ratio_sb_R_Aduana', 'msb_c', 'msb_fide', 'msb_ot'], inplace=True)

sb_ratios_red.info()
sb_ratios_red.head(26)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  26 non-null     object 
 1   SECTOR          26 non-null     object 
 2   PLIEGO          26 non-null     object 
 3   FUENTE          26 non-null     object 
 4   RUBRO           26 non-null     object 
 5   ratio_sb_fed    25 non-null     float64
 6   ratio_sb_ra     2 non-null      float64
dtypes: float64(2), object(5)
memory usage: 1.6+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,ratio_sb_fed,ratio_sb_ra
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.570227,NaN
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.022269,NaN
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.295271,NaN
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.065611,7.365332e-09
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.377726,NaN
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.069173,NaN
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.044568,NaN
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.129743,NaN
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.601542,NaN
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0.063413,NaN


In [26]:
outputFile = os.path.join(r'Y:\1) NUEVO compartido DAPT\Financiamiento_Data\PMG 2025-2027\Outputs','sb_ratios_red'+ fecha_corte+ ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    sb_ratios_red.to_excel(ew, sheet_name="Ratios", index = False)
#%%

In [27]:
ratios_sb_c = pd.merge(ratios_preliminar_red, sb_ratios_red, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], how='outer', validate = 'm:1')
postmerge(ratios_sb_c)
ratios_sb_c.info()
ratios_sb_c.head(88)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NIVEL_GOBIERNO       98 non-null     object 
 1   SECTOR               98 non-null     object 
 2   PLIEGO               98 non-null     object 
 3   FUENTE               98 non-null     object 
 4   RUBRO                98 non-null     object 
 5   CATEGORIA_GTO        98 non-null     object 
 6   GENERICA             98 non-null     object 
 7   mean_ratio_C_SC_RM   81 non-null     float64
 8   mean_ratio_R_Aduana  7 non-null      float64
 9   ratio_f_c            98 non-null     float64
 10  comp                 98 non-null     float64
 11  ratio_f_ra           98 non-null     float64
 12  compra               98 non-null     float64
 13  ratio_sb_fed         95 non-null     float64
 14  ratio_sb_ra          11 non-null     float64
dtypes: float64(8), object(7)
memory usage: 11.

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,mean_ratio_C_SC_RM,mean_ratio_R_Aduana,ratio_f_c,comp,ratio_f_ra,compra,ratio_sb_fed,ratio_sb_ra
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000000,NaN,0.000000,0.0,0.0,1.0,0.570227,NaN
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,NaN,1.000000,0.0,1.0,1.0,0.570227,NaN
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.105649,NaN,0.093246,0.0,0.0,1.0,0.022269,NaN
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,5. OTROS GASTOS,NaN,NaN,0.000000,0.0,0.0,1.0,0.022269,NaN
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.336875,NaN,0.297327,0.0,0.0,1.0,0.022269,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,NaN,NaN,0.000000,0.0,0.0,1.0,0.245124,NaN
84,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.865462,NaN,0.865462,0.0,1.0,1.0,0.245124,NaN
85,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.114724,NaN,0.108210,0.0,0.0,1.0,0.117838,NaN
86,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.143140,NaN,0.135013,0.0,0.0,1.0,0.117838,NaN


In [28]:
outputFile = os.path.join(r'Y:\1) NUEVO compartido DAPT\Financiamiento_Data\PMG 2025-2027\Outputs','ratios_sb_c'+ fecha_corte+ ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    ratios_sb_c.to_excel(ew, sheet_name="Ratios", index = False)
#%%

In [29]:
bd_sd.info()
bd_sd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOB       2827 non-null   object 
 1   SECTOR          2827 non-null   object 
 2   PLIEGO          2827 non-null   object 
 3   EJECUTORA       2827 non-null   object 
 4   FUNCION         2827 non-null   object 
 5   FUENTE          2827 non-null   object 
 6   RUBRO           2827 non-null   object 
 7   CATEGORIA_GTO   2827 non-null   object 
 8   GENERICA        2827 non-null   object 
 9   SUBGENERICA     2827 non-null   object 
 10  PIA_2018        2827 non-null   int64  
 11  PIA_2019        2827 non-null   int64  
 12  PIA_2020        2827 non-null   int64  
 13  PIA_2021        2827 non-null   int64  
 14  PIA_2022        2827 non-null   int64  
 15  PIA_2023        2827 non-null   int64  
 16  PIA_2024        2827 non-null   int64  
 17  PIM_2018        2827 non-null   i

,NIVEL_GOB,SECTOR,PLIEGO,EJECUTORA,FUNCION,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,SUBGENERICA,...,PMG_2025,PMG_2026,PMG_2027,PMG_2028,PMG_2029,PMG_2030,PMG_2031,PMG_2032,PMG_2033,PMG_2034
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,"03. PLANEAMIENTO, GESTION Y RESERVA DE CONTING...",5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,2. INTERESES DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,3. COMISIONES Y OTROS GASTOS DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,001. SEDE APURIMAC,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,6418316,6418316,19470737,13052420,13052420,13052420,13052420,13052420,13052420,13052420


## Servicio de la deuda financiada con RD

In [30]:
serv_deuda = bd_sd.copy()

serv_deuda.rename(columns={'NIVEL_GOB' : 'NIVEL_GOBIERNO'}, inplace=True)
serv_deuda= serv_deuda[serv_deuda['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
serv_deuda= serv_deuda[serv_deuda['FUENTE'] == '5. RECURSOS DETERMINADOS']
serv_deuda = serv_deuda[serv_deuda['RUBRO'].str[:2] == '18']
serv_deuda= serv_deuda[serv_deuda['GENERICA'] == '8. SERVICIO DE LA DEUDA PUBLICA']
# Calcular la suma de las columnas PMG_2025, PMG_2026 y PMG_2027 agrupadas por PLIEGO y GENERICA
serv_deuda = serv_deuda.groupby(['NIVEL_GOBIERNO','PLIEGO', 'GENERICA'])[['PMG_2025', 'PMG_2026', 'PMG_2027']].sum().reset_index()

# Renombrar las columnas PMG_2025, PMG_2026 y PMG_2027 a SD_2025_T, SD_2026_T y SD_2027_T respectivamente
serv_deuda = serv_deuda.rename(columns={'PMG_2025': 'SD_2025_T', 'PMG_2026': 'SD_2026_T', 'PMG_2027': 'SD_2027_T'})

# Reemplazar los valores nulos por 0 en las columnas especificadas
serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']] = serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']].fillna(0)

# Calcular la suma de las columnas SD_2025_T, SD_2026_T y SD_2027_T y guardarla en una nueva columna llamada SD_PMM
serv_deuda['SD_PMM'] = serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']].sum(axis=1)

# Eliminar filas donde SD_PMM es igual a 0 o es nulo
serv_deuda = serv_deuda.dropna(subset=['SD_PMM'])
serv_deuda = serv_deuda[serv_deuda['SD_PMM'].fillna(0) != 0].reset_index(drop=True)

serv_deuda.info()
serv_deuda.head(16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  16 non-null     object
 1   PLIEGO          16 non-null     object
 2   GENERICA        16 non-null     object
 3   SD_2025_T       16 non-null     int64 
 4   SD_2026_T       16 non-null     int64 
 5   SD_2027_T       16 non-null     int64 
 6   SD_PMM          16 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 1.0+ KB


,NIVEL_GOBIERNO,PLIEGO,GENERICA,SD_2025_T,SD_2026_T,SD_2027_T,SD_PMM
0,2. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,8. SERVICIO DE LA DEUDA PUBLICA,6435953,7323825,19612855,33372633
1,2. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,8. SERVICIO DE LA DEUDA PUBLICA,168751467,422490872,55523384,646765723
2,2. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,8. SERVICIO DE LA DEUDA PUBLICA,27253482,47701872,23878471,98833825
3,2. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,8. SERVICIO DE LA DEUDA PUBLICA,33896775,32436416,31559441,97892632
4,2. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,8. SERVICIO DE LA DEUDA PUBLICA,8522051,4787244,0,13309295
5,2. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,8. SERVICIO DE LA DEUDA PUBLICA,0,1810681,0,1810681
6,2. GOBIERNOS REGIONALES,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,8. SERVICIO DE LA DEUDA PUBLICA,9143347,33612243,18669586,61425176
7,2. GOBIERNOS REGIONALES,453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO,8. SERVICIO DE LA DEUDA PUBLICA,9266852,127658,127658,9522168
8,2. GOBIERNOS REGIONALES,455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQ...,8. SERVICIO DE LA DEUDA PUBLICA,2510672,2508183,2505694,7524549
9,2. GOBIERNOS REGIONALES,456. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PASCO,8. SERVICIO DE LA DEUDA PUBLICA,4400574,4062068,3177425,11640067


In [31]:
bd_rubro18.info()
bd_rubro18.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NIVEL_GOBIERNO    1068 non-null   object
 1   SECTOR            1068 non-null   object
 2   PLIEGO            1068 non-null   object
 3   EJECUTORA         1068 non-null   object
 4   FUENTE            1068 non-null   object
 5   RUBRO             1068 non-null   object
 6   tipo_gen          1068 non-null   object
 7   ETIQUETA_RD       1068 non-null   object
 8   RECAUDACION_2024  1068 non-null   int64 
 9   RECAUDACION_2025  1068 non-null   int64 
 10  RECAUDACION_2026  1068 non-null   int64 
 11  RECAUDACION_2027  1068 non-null   int64 
 12  PMG_2025_gg3_1    1068 non-null   int64 
 13  PMG_2025_gg2_1    1068 non-null   int64 
 14  PMG_2025_gg6_2    1068 non-null   int64 
 15  PMG_2025_gg7_2    1068 non-null   int64 
 16  PMG_2026_gg3_1    1068 non-null   int64 
 17  PMG_2026_gg2_1

,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,FUENTE,RUBRO,tipo_gen,ETIQUETA_RD,RECAUDACION_2024,RECAUDACION_2025,...,PMG_2025_gg6_2,PMG_2025_gg7_2,PMG_2026_gg3_1,PMG_2026_gg2_1,PMG_2026_gg6_2,PMG_2026_gg7_2,PMG_2027_gg3_1,PMG_2027_gg2_1,PMG_2027_gg6_2,PMG_2027_gg7_2
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,005. PROAMAZONAS,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3. Canon_SB,308457,500,...,0,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3. Canon_SB,85000000,85000000,...,0,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,003. SUB REGION PACIFICO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3. Canon_SB,9850000,9850000,...,0,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,007. PROYECTO ESPECIAL CHINECAS,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3. Canon_SB,7494292,1300000,...,0,0,0,0,0,0,0,0,0,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,200. TRANSPORTES ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3. Canon_SB,7164965,5000000,...,0,0,0,0,0,0,0,0,0,0


## Base ingresos Saldos de balance RD 

In [32]:
PIASB2527 = bd_rubro18.copy()

# Mantener solo las filas donde la columna tipo_gen es igual a "1. SALDOS DE BALANCE"
PIASB2527 = PIASB2527[PIASB2527['tipo_gen'] == "1. SALDOS DE BALANCE"]
PIASB2527= PIASB2527[PIASB2527['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
PIASB2527['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"

# Renombrar las columnas RECAUDACION_2025, RECAUDACION_2026 y RECAUDACION_2027 a PMG_2025, PMG_2026 y PMG_2027 respectivamente
PIASB2527.rename(columns={'RECAUDACION_2025': 'PMG_2025', 'RECAUDACION_2026': 'PMG_2026', 'RECAUDACION_2027': 'PMG_2027'}, inplace=True)

# Agrupar por las columnas especificadas y sumar las columnas PMG_2025, PMG_2026 y PMG_2027
PIASB2527 = PIASB2527.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'tipo_gen']).agg({'PMG_2025': 'sum', 'PMG_2026': 'sum', 'PMG_2027': 'sum', 
                                                                                                             'PMG_2025_gg3_1': 'sum', 'PMG_2025_gg2_1': 'sum', 'PMG_2025_gg6_2':'sum',	'PMG_2025_gg7_2': 'sum', 
                                                                                                             'PMG_2026_gg3_1': 'sum', 'PMG_2026_gg2_1': 'sum', 'PMG_2026_gg6_2': 'sum', 'PMG_2026_gg7_2': 'sum', 
                                                                                                             'PMG_2027_gg3_1': 'sum', 'PMG_2027_gg2_1': 'sum', 'PMG_2027_gg6_2': 'sum', 'PMG_2027_gg7_2': 'sum'}).reset_index()

# Renombrar las columnas PMG_2025, PMG_2026 y PMG_2027 a SB_2025, SB_2026, SB_2027 y tipo_gen a SaldoBalance respectivamente
PIASB2527.rename(columns={'PMG_2025': 'SB_2025', 'PMG_2026': 'SB_2026', 'PMG_2027': 'SB_2027', 'tipo_gen': 'SaldoBalance'}, inplace=True)

# Redondear los valores en las columnas SB_2025, SB_2026 y SB_2027
PIASB2527['SB_2025'] = PIASB2527['SB_2025'].round()
PIASB2527['SB_2026'] = PIASB2527['SB_2026'].round()
PIASB2527['SB_2027'] = PIASB2527['SB_2027'].round()

PIASB2527.info()
PIASB2527.head(24)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  24 non-null     object
 1   SECTOR          24 non-null     object
 2   PLIEGO          24 non-null     object
 3   FUENTE          24 non-null     object
 4   RUBRO           24 non-null     object
 5   SaldoBalance    24 non-null     object
 6   SB_2025         24 non-null     int64 
 7   SB_2026         24 non-null     int64 
 8   SB_2027         24 non-null     int64 
 9   PMG_2025_gg3_1  24 non-null     int64 
 10  PMG_2025_gg2_1  24 non-null     int64 
 11  PMG_2025_gg6_2  24 non-null     int64 
 12  PMG_2025_gg7_2  24 non-null     int64 
 13  PMG_2026_gg3_1  24 non-null     int64 
 14  PMG_2026_gg2_1  24 non-null     int64 
 15  PMG_2026_gg6_2  24 non-null     int64 
 16  PMG_2026_gg7_2  24 non-null     int64 
 17  PMG_2027_gg3_1  24 non-null     int64 
 18  PMG_2027_gg2

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,SaldoBalance,SB_2025,SB_2026,SB_2027,PMG_2025_gg3_1,...,PMG_2025_gg6_2,PMG_2025_gg7_2,PMG_2026_gg3_1,PMG_2026_gg2_1,PMG_2026_gg6_2,PMG_2026_gg7_2,PMG_2027_gg3_1,PMG_2027_gg2_1,PMG_2027_gg6_2,PMG_2027_gg7_2
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,500,500,500,0,...,0,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,101150000,101150000,101150000,0,...,0,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,2174700,2555000,3065000,0,...,0,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,16000000,16000000,16000000,4367603,...,0,0,1367603,0,0,0,0,0,0,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,6044986,6212450,6618876,2808026,...,20000,2000000,3802194,0,0,2100000,3064000,0,0,3000000
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,43934200,41706169,42126787,0,...,0,0,0,0,0,0,0,0,0,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,100000,100000,100000,0,...,0,0,0,0,0,0,0,0,0,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,1277834,1182954,1178334,268700,...,919500,0,228820,30000,860000,0,227200,30000,860000,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,18000000,17000000,16000000,266000,...,0,0,268000,0,0,0,270000,0,0,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,3270530,3570530,3770530,10530,...,0,0,10530,0,0,0,10530,0,0,0


## Base de Estimaciones a los recursos de RD + Saldos de Balance  

In [33]:
R18= bd_R18.copy()
R18=R18[['NIVEL DE GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'ESPECIFICA DET', '2025', '2026', '2027']]
R18.rename(columns={'NIVEL DE GOBIERNO': 'NIVEL_GOBIERNO'}, inplace=True)
R18= R18[R18['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
R18= R18[R18['FUENTE'] == '5. RECURSOS DETERMINADOS']
R18= R18[R18['RUBRO'] == '18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES']
R18 = R18.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'ESPECIFICA DET']).agg({'2025':'sum', '2026':'sum', '2027':'sum'}).reset_index()
# Eliminar filas duplicadas en el dataframe
R18.drop_duplicates(inplace=True)
R18.info()
R18.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  106 non-null    object 
 1   SECTOR          106 non-null    object 
 2   PLIEGO          106 non-null    object 
 3   FUENTE          106 non-null    object 
 4   RUBRO           106 non-null    object 
 5   ESPECIFICA DET  106 non-null    object 
 6   2025            106 non-null    int64  
 7   2026            106 non-null    float64
 8   2027            106 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 7.6+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,ESPECIFICA DET,2025,2026,2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. PARTICIPACION POR ELIMINACION DE EXONERACIONES,11457628,11686781.0,11920517.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. CANON FORESTAL,4539,4539.0,4539.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. REGALIAS MINERAS,33393815,33921892.0,32792202.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",3. CANON MINERO,179283230,169315302.0,163659824.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. CANON HIDROENERGETICO,2644781,2560287.0,2560287.0


In [34]:
bd_R18_25= R18.copy()
bd_R18_25 = bd_R18_25.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2025']).reset_index().rename_axis(None)
bd_R18_25.columns = [f'{i}{j}' for i, j in bd_R18_25.columns]
bd_R18_25.rename(columns={'20253. CANON MINERO':'CANON MINERO', '20251. CANON PETROLERO':'CANON PETROLERO', '20252. SOBRECANON PETROLERO':'SOBRECANON PETROLERO', 
                    '20254. CANON GASIFERO':'CANON GASIFERO', '20255. CANON HIDROENERGETICO':'CANON HIDROENER-GETICO', '20256. CANON PESQUERO':'CANON PESQUERO', 
                    '20257. CANON FORESTAL': 'CANON FORESTAL',  '20251. RENTA DE ADUANAS':'RENTA DE ADUANAS  1/', '20252. REGALIAS FOCAM': 'FOCAM',  
                   '20251. PARTICIPACION POR ELIMINACION DE EXONERACIONES':'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', '20251. REGALIAS MINERAS':'REGALIAS MINERAS',
                   '20252. REGALIA DE LA ACTIVIDAD MINERA':'REGALIA CONTRACTUAL'}, inplace=True)

bd_R18_25=bd_R18_25[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']]
bd_R18_25.fillna(0, inplace=True)

variables = ['CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']
for var in variables:
    bd_R18_25[var] = bd_R18_25[var].round()
    bd_R18_25[var] =  bd_R18_25[var].astype(int)
    
bd_R18_25.info()
bd_R18_25.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11457628,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",179283230,0,0,0,2644781,5184970,7567,0,0,0,33393815,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",29331974,0,0,0,0,0,660,0,0,0,652114,40628249
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",146038247,0,0,0,2740044,358072,1575,0,0,0,21119112,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",8075242,0,0,0,0,0,4510,0,41125059,0,2312625,0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",21453894,0,0,0,1111998,0,24408,0,0,0,6927159,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",39757017,0,0,566924771,4922948,0,9553,0,0,0,7761361,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",179124,0,0,0,16272519,0,0,0,22611726,0,893419,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,10198,0,0,0,318032,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",93170699,0,0,0,0,2042665,848,0,34927453,0,28610352,0


In [35]:
bd_R18_26= R18.copy()
bd_R18_26 = bd_R18_26.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2026']).reset_index().rename_axis(None)
bd_R18_26.columns = [f'{i}{j}' for i, j in bd_R18_26.columns]
bd_R18_26.rename(columns={'20263. CANON MINERO':'CANON MINERO', '20261. CANON PETROLERO':'CANON PETROLERO', '20262. SOBRECANON PETROLERO':'SOBRECANON PETROLERO', 
                    '20264. CANON GASIFERO':'CANON GASIFERO', '20265. CANON HIDROENERGETICO':'CANON HIDROENER-GETICO', '20266. CANON PESQUERO':'CANON PESQUERO', 
                    '20267. CANON FORESTAL': 'CANON FORESTAL',  '20261. RENTA DE ADUANAS':'RENTA DE ADUANAS  1/', '20262. REGALIAS FOCAM': 'FOCAM',  
                   '20261. PARTICIPACION POR ELIMINACION DE EXONERACIONES':'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', '20261. REGALIAS MINERAS':'REGALIAS MINERAS',
                   '20262. REGALIA DE LA ACTIVIDAD MINERA':'REGALIA CONTRACTUAL'}, inplace=True)

bd_R18_26=bd_R18_26[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']]
bd_R18_26.fillna(0, inplace=True)
variables = ['CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']
for var in variables:
    bd_R18_26[var] = bd_R18_26[var].round()
    bd_R18_26[var] =  bd_R18_26[var].astype(int)
bd_R18_26.info()
bd_R18_26.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11686781,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",169315302,0,0,0,2560287,5956344,7567,0,0,0,33921892,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",28138272,0,0,0,0,0,660,0,0,0,766578,37470685
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",133380542,0,0,0,2651966,403993,1575,0,0,0,20100413,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7010086,0,0,0,0,0,4510,0,39361870,0,2938560,0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19614748,0,0,0,1112250,0,24408,0,0,0,8092719,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",35955648,0,0,557713491,4945041,0,9553,0,0,0,8844605,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",117452,0,0,0,18187706,0,0,0,21642275,0,851436,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,10198,0,0,0,0,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",81564618,0,0,0,0,2281264,848,0,33429980,0,29828675,0


In [36]:
bd_R18_27= R18.copy()
bd_R18_27 = bd_R18_27.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2027']).reset_index().rename_axis(None)
bd_R18_27.columns = [f'{i}{j}' for i, j in bd_R18_27.columns]
bd_R18_27.rename(columns={'20273. CANON MINERO':'CANON MINERO', '20271. CANON PETROLERO':'CANON PETROLERO', '20272. SOBRECANON PETROLERO':'SOBRECANON PETROLERO', 
                    '20274. CANON GASIFERO':'CANON GASIFERO', '20275. CANON HIDROENERGETICO':'CANON HIDROENER-GETICO', '20276. CANON PESQUERO':'CANON PESQUERO', 
                    '20277. CANON FORESTAL': 'CANON FORESTAL',  '20271. RENTA DE ADUANAS':'RENTA DE ADUANAS  1/', '20272. REGALIAS FOCAM': 'FOCAM',  
                   '20271. PARTICIPACION POR ELIMINACION DE EXONERACIONES':'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', '20271. REGALIAS MINERAS':'REGALIAS MINERAS',
                   '20272. REGALIA DE LA ACTIVIDAD MINERA':'REGALIA CONTRACTUAL'}, inplace=True)

bd_R18_27=bd_R18_27[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']]
bd_R18_27.fillna(0, inplace=True)
variables = ['CANON MINERO', 'CANON PETROLERO', 'SOBRECANON PETROLERO', 'CANON GASIFERO', 'CANON HIDROENER-GETICO', 'CANON PESQUERO',
        'CANON FORESTAL', 'RENTA DE ADUANAS  1/', 'FOCAM', 'ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS', 'REGALIAS MINERAS', 'REGALIA CONTRACTUAL']
for var in variables:
    bd_R18_27[var] = bd_R18_27[var].round()
    bd_R18_27[var] =  bd_R18_27[var].astype(int)
bd_R18_27.info()
bd_R18_27.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11920517,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",163659824,0,0,0,2560287,6603348,7567,0,0,0,32792202,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",27358323,0,0,0,0,0,660,0,0,0,674733,35065736
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",126286870,0,0,0,2651966,437098,1575,0,0,0,20352070,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6245784,0,0,0,0,0,4510,0,37454250,0,2465432,0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",18240383,0,0,0,1112340,0,24408,0,0,0,7150811,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",33953162,0,0,543188594,5151266,0,9553,0,0,0,8040799,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",78029,0,0,0,14721126,0,0,0,20593411,0,829742,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,10198,0,0,0,205703,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",76942532,0,0,0,0,2464619,848,0,31809840,0,28541734,0


In [37]:
R18_25 = bd_R18_25.copy() 
R18_26 = bd_R18_26.copy() 
R18_27 = bd_R18_27.copy() 

In [38]:
R18_25.info()
R18_25.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11457628,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",179283230,0,0,0,2644781,5184970,7567,0,0,0,33393815,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",29331974,0,0,0,0,0,660,0,0,0,652114,40628249
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",146038247,0,0,0,2740044,358072,1575,0,0,0,21119112,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",8075242,0,0,0,0,0,4510,0,41125059,0,2312625,0


In [39]:
R18_26.info()
R18_26.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11686781,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",169315302,0,0,0,2560287,5956344,7567,0,0,0,33921892,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",28138272,0,0,0,0,0,660,0,0,0,766578,37470685
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",133380542,0,0,0,2651966,403993,1575,0,0,0,20100413,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7010086,0,0,0,0,0,4510,0,39361870,0,2938560,0


In [40]:
R18_27.info()
R18_27.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   NIVEL_GOBIERNO                                       26 non-null     object
 1   SECTOR                                               26 non-null     object
 2   PLIEGO                                               26 non-null     object
 3   FUENTE                                               26 non-null     object
 4   RUBRO                                                26 non-null     object
 5   CANON MINERO                                         26 non-null     int32 
 6   CANON PETROLERO                                      26 non-null     int32 
 7   SOBRECANON PETROLERO                                 26 non-null     int32 
 8   CANON GASIFERO                                       26 non-null     int32 
 9   C

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CANON MINERO,CANON PETROLERO,SOBRECANON PETROLERO,CANON GASIFERO,CANON HIDROENER-GETICO,CANON PESQUERO,CANON FORESTAL,RENTA DE ADUANAS 1/,FOCAM,ELIMINACION EXONERACIONES E INCENTIVOS TRIBUTARIOS,REGALIAS MINERAS,REGALIA CONTRACTUAL
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,4539,0,0,11920517,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",163659824,0,0,0,2560287,6603348,7567,0,0,0,32792202,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",27358323,0,0,0,0,0,660,0,0,0,674733,35065736
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",126286870,0,0,0,2651966,437098,1575,0,0,0,20352070,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",6245784,0,0,0,0,0,4510,0,37454250,0,2465432,0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",18240383,0,0,0,1112340,0,24408,0,0,0,7150811,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",33953162,0,0,543188594,5151266,0,9553,0,0,0,8040799,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",78029,0,0,0,14721126,0,0,0,20593411,0,829742,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",0,0,0,0,0,0,10198,0,0,0,205703,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",76942532,0,0,0,0,2464619,848,0,31809840,0,28541734,0


In [41]:
def PIA2527(df,year):
    
    df.rename(columns={"CANON MINERO": "CANONMINERO",
                       "CANON PETROLERO": "CANONPETROLERO",
                       "SOBRECANON PETROLERO": "SOBRECANONPETROLERO",
                       "CANON GASIFERO": "CANONGASIFERO",
                       "CANON HIDROENER-GETICO": "CANONHIDROENERGETICO",
                       "CANON PESQUERO": "CANONPESQUERO",
                       "CANON FORESTAL": "CANONFORESTAL",
                       "REGALIAS MINERAS": "REGALIASMINERAS",
                       "ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS": "ELIMINACIONEXONERACIONESEINC",
                       "REGALIA CONTRACTUAL": "REGALIACONTRACTUAL",
                       "FOCAM ":"FOCAM",
                       'RENTA DE ADUANAS  1/':'RENTADEADUANAS1'}, inplace=True)
    
    variables = ["CANONMINERO", "CANONPETROLERO", "SOBRECANONPETROLERO","CANONGASIFERO", "CANONHIDROENERGETICO", "CANONPESQUERO", 
                "CANONFORESTAL", 'RENTADEADUANAS1', 'FOCAM', 'REGALIASMINERAS', 'ELIMINACIONEXONERACIONESEINC', 'REGALIACONTRACTUAL']
    
    for var in variables: 
        df[var] = df[var].astype(str)
        df[var] = df[var].str.replace(" ", '0')
        df[var] = df[var].fillna('0')
        df[var] =  df[var].astype(int)
    
    # Filtrar por "2. GOBIERNOS REGIONALES"
    df = df[df["NIVEL_GOBIERNO"] == "2. GOBIERNOS REGIONALES"]
    

    # Convertir columnas a números y reemplazar valores no válidos
    df["CANONMINERO"] = pd.to_numeric(df["CANONMINERO"], errors="coerce")
    df["REGALIACONTRACTUAL"] = pd.to_numeric(df["REGALIACONTRACTUAL"], errors="coerce")
    df.dropna(subset=["CANONMINERO", "REGALIACONTRACTUAL"], inplace=True)

    # Sumarizar y colapsar
    df = df.groupby("PLIEGO").agg({'NIVEL_GOBIERNO':'first', "CANONMINERO": "sum", "CANONPETROLERO": "sum", "SOBRECANONPETROLERO": "sum", "CANONGASIFERO": "sum", 
                                   "CANONHIDROENERGETICO": "sum", "CANONPESQUERO": "sum", "CANONFORESTAL": "sum", "FOCAM": "sum", 
                                   "REGALIASMINERAS": "sum", "ELIMINACIONEXONERACIONESEINC": "sum", "REGALIACONTRACTUAL": "sum", 
                                   'RENTADEADUANAS1' : 'sum', "FOCAM":'sum'}).reset_index()

    # Redondear variables específicas
    #vars_to_round = ["CANONMINERO", "CANONPETROLERO", "SOBRECANONPETROLERO", "CANONGASIFERO", 
     #                "CANONHIDROENERGETICO", "CANONPESQUERO", "CANONFORESTAL", 'RENTADEADUANAS1', "FOCAM", 
      #               "REGALIASMINERAS", "ELIMINACIONEXONERACIONESEINC", "REGALIACONTRACTUAL"]
    #df[vars_to_round] = df[vars_to_round].applymap(lambda x: int(x) if pd.notnull(x) else x)

    # Renombrar columnas
    df.rename(columns={"FOCAM": "Regalias_FOCAM"}, inplace=True)

    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['Canon_Sobrecanon'] = df[["CANONMINERO", "CANONPETROLERO", "SOBRECANONPETROLERO",
                                  "CANONGASIFERO", "CANONHIDROENERGETICO", "CANONPESQUERO", "CANONFORESTAL"]].sum(axis=1)
    
    df["Res_Direct"] = df[["CANONMINERO", "CANONPETROLERO", "SOBRECANONPETROLERO", "CANONGASIFERO", 
                            "CANONHIDROENERGETICO", "CANONPESQUERO", "CANONFORESTAL", "Regalias_FOCAM", 'RENTADEADUANAS1',
                            "REGALIASMINERAS", "ELIMINACIONEXONERACIONESEINC", "REGALIACONTRACTUAL"]].sum(axis=1)
    ##verificar que Res_Direct sean apropiadas las variables que se suman ???.
    
    # Renombrar columnas
    df.rename(columns={"REGALIASMINERAS": "Regalias_Mineras", 
                       "ELIMINACIONEXONERACIONESEINC": "Participación_Elim_Exo",
                       "REGALIACONTRACTUAL": "Regalias_Contractuales", 
                       'RENTADEADUANAS1' : 'Participación_Renta_Aduana'}, inplace=True)

    # Eliminar columnas innecesarias
    df.drop(columns=['CANONMINERO', "CANONPETROLERO", "SOBRECANONPETROLERO", "CANONGASIFERO", 
                     "CANONHIDROENERGETICO", "CANONPESQUERO", "CANONFORESTAL"], inplace=True)

    # Fusionar con otro DataFrame
    #*Incluimos saldos de balance del mòdulo de programacion de ingresos
    df = df.merge(PIASB2527[["PLIEGO", f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SB_{year}"].fillna(0, inplace=True)
    df[f"PMG_{year}_gg3_1"].fillna(0, inplace=True)
    df[f"PMG_{year}_gg6_2"].fillna(0, inplace=True)

    # Calcular PIA_BASE
    df["PIA_BASE"] = df["Res_Direct"].fillna(0) + df[f"SB_{year}"].fillna(0)

    # Agregar columna AÑO
    df["AÑO"] = year
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES"
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"
    # Renombrar columnas SB*
    df.rename(columns={"PIA_BASE":f"PIA_BASE_{year}"}, inplace=True)
    return df

In [42]:
PIA_2025_base24 = PIA2527(R18_25, 2025)
PIA_2025_base24.info()
PIA_2025_base24.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PLIEGO                      26 non-null     object 
 1   NIVEL_GOBIERNO              26 non-null     object 
 2   Regalias_FOCAM              26 non-null     int32  
 3   Regalias_Mineras            26 non-null     int32  
 4   Participación_Elim_Exo      26 non-null     int32  
 5   Regalias_Contractuales      26 non-null     int32  
 6   Participación_Renta_Aduana  26 non-null     int32  
 7   Canon_Sobrecanon            26 non-null     int64  
 8   Res_Direct                  26 non-null     int64  
 9   SB_2025                     26 non-null     float64
 10  PMG_2025_gg3_1              26 non-null     float64
 11  PMG_2025_gg6_2              26 non-null     float64
 12  PIA_BASE_2025               26 non-null     float64
 13  AÑO                         26 non-nu

,PLIEGO,NIVEL_GOBIERNO,Regalias_FOCAM,Regalias_Mineras,Participación_Elim_Exo,Regalias_Contractuales,Participación_Renta_Aduana,Canon_Sobrecanon,Res_Direct,SB_2025,PMG_2025_gg3_1,PMG_2025_gg6_2,PIA_BASE_2025,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,0,0,11457628,0,0,4539,11462167,500.0,0.0,0.0,11462667.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,0,33393815,0,0,0,187120548,220514363,101150000.0,0.0,0.0,321664363.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,0,652114,0,40628249,0,29332634,70612997,2174700.0,0.0,0.0,72787697.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,0,21119112,0,0,0,149137938,170257050,16000000.0,4367603.0,0.0,186257050.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,41125059,2312625,0,0,0,8079752,51517436,6044986.0,2808026.0,20000.0,57562422.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


In [43]:
PIA_2026_base24 = PIA2527(R18_26, 2026)
PIA_2026_base24.info()
PIA_2026_base24.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PLIEGO                      26 non-null     object 
 1   NIVEL_GOBIERNO              26 non-null     object 
 2   Regalias_FOCAM              26 non-null     int32  
 3   Regalias_Mineras            26 non-null     int32  
 4   Participación_Elim_Exo      26 non-null     int32  
 5   Regalias_Contractuales      26 non-null     int32  
 6   Participación_Renta_Aduana  26 non-null     int32  
 7   Canon_Sobrecanon            26 non-null     int64  
 8   Res_Direct                  26 non-null     int64  
 9   SB_2026                     26 non-null     float64
 10  PMG_2026_gg3_1              26 non-null     float64
 11  PMG_2026_gg6_2              26 non-null     float64
 12  PIA_BASE_2026               26 non-null     float64
 13  AÑO                         26 non-nu

,PLIEGO,NIVEL_GOBIERNO,Regalias_FOCAM,Regalias_Mineras,Participación_Elim_Exo,Regalias_Contractuales,Participación_Renta_Aduana,Canon_Sobrecanon,Res_Direct,SB_2026,PMG_2026_gg3_1,PMG_2026_gg6_2,PIA_BASE_2026,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,0,0,11686781,0,0,4539,11691320,500.0,0.0,0.0,11691820.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,0,33921892,0,0,0,177839500,211761392,101150000.0,0.0,0.0,312911392.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,0,766578,0,37470685,0,28138932,66376195,2555000.0,0.0,0.0,68931195.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,0,20100413,0,0,0,136438076,156538489,16000000.0,1367603.0,0.0,172538489.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,39361870,2938560,0,0,0,7014596,49315026,6212450.0,3802194.0,0.0,55527476.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
5,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2. GOBIERNOS REGIONALES,0,8092719,0,0,0,20751406,28844125,0.0,0.0,0.0,28844125.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
6,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,2. GOBIERNOS REGIONALES,0,8844605,0,0,0,598623733,607468338,41706169.0,0.0,0.0,649174507.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
7,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,21642275,851436,0,0,0,18305158,40798869,100000.0,0.0,0.0,40898869.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
8,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,0,0,0,0,0,10198,10198,1182954.0,228820.0,860000.0,1193152.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
9,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,2. GOBIERNOS REGIONALES,33429980,29828675,0,0,0,83846730,147105385,17000000.0,268000.0,0.0,164105385.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


In [44]:
PIA_2027_base24 = PIA2527(R18_27, 2027)
PIA_2027_base24.info()
PIA_2027_base24.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PLIEGO                      26 non-null     object 
 1   NIVEL_GOBIERNO              26 non-null     object 
 2   Regalias_FOCAM              26 non-null     int32  
 3   Regalias_Mineras            26 non-null     int32  
 4   Participación_Elim_Exo      26 non-null     int32  
 5   Regalias_Contractuales      26 non-null     int32  
 6   Participación_Renta_Aduana  26 non-null     int32  
 7   Canon_Sobrecanon            26 non-null     int64  
 8   Res_Direct                  26 non-null     int64  
 9   SB_2027                     26 non-null     float64
 10  PMG_2027_gg3_1              26 non-null     float64
 11  PMG_2027_gg6_2              26 non-null     float64
 12  PIA_BASE_2027               26 non-null     float64
 13  AÑO                         26 non-nu

,PLIEGO,NIVEL_GOBIERNO,Regalias_FOCAM,Regalias_Mineras,Participación_Elim_Exo,Regalias_Contractuales,Participación_Renta_Aduana,Canon_Sobrecanon,Res_Direct,SB_2027,PMG_2027_gg3_1,PMG_2027_gg6_2,PIA_BASE_2027,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,0,0,11920517,0,0,4539,11925056,500.0,0.0,0.0,11925556.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,0,32792202,0,0,0,172831026,205623228,101150000.0,0.0,0.0,306773228.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,0,674733,0,35065736,0,27358983,63099452,3065000.0,0.0,0.0,66164452.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,0,20352070,0,0,0,129377509,149729579,16000000.0,0.0,0.0,165729579.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,37454250,2465432,0,0,0,6250294,46169976,6618876.0,3064000.0,0.0,52788852.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
5,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2. GOBIERNOS REGIONALES,0,7150811,0,0,0,19377131,26527942,0.0,0.0,0.0,26527942.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
6,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,2. GOBIERNOS REGIONALES,0,8040799,0,0,0,582302575,590343374,42126787.0,0.0,0.0,632470161.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
7,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,20593411,829742,0,0,0,14799155,36222308,100000.0,0.0,0.0,36322308.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
8,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,0,205703,0,0,0,10198,215901,1178334.0,227200.0,860000.0,1394235.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
9,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,2. GOBIERNOS REGIONALES,31809840,28541734,0,0,0,79407999,139759573,16000000.0,270000.0,0.0,155759573.0,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


# 3. Propuesta nueva RD(modif) PMG 2025 -2027

## DISTRIBUCIÓN 2025, 2026 y 2027

### Generar PIA Base de ingresos sobre el cual se hace la distribución de gastos  (SB y Flujo)

In [45]:
PIA_2025_bd24= PIA_2025_base24.copy()
PIA_2026_bd24= PIA_2026_base24.copy()
PIA_2027_bd24= PIA_2027_base24.copy()

### 7. SERVICIO DE LA DEUDA

In [46]:
def PIASBF(df,year):
    
    variables = ['Canon_Sobrecanon', 'Regalias_Mineras', 'Regalias_FOCAM', 'Participación_Elim_Exo', 'Participación_Renta_Aduana', 'Regalias_Contractuales']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
        
    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['C_SC_RM'] = df[["Canon_Sobrecanon", "Regalias_Mineras"]].sum(axis=1) 
    df['OR'] = df[["Regalias_FOCAM", "Participación_Elim_Exo", "Regalias_Contractuales" ]].sum(axis=1)
    df['RA'] = df['Participación_Renta_Aduana']
    
    # Eliminar columnas innecesarias
    df.drop(columns=['Regalias_Mineras', "Regalias_Contractuales", "Regalias_FOCAM", "Participación_Renta_Aduana", 
                     "Participación_Elim_Exo", "Canon_Sobrecanon"], inplace=True)
    df = df[['AÑO', 'PLIEGO', 'C_SC_RM', 'RA', 'OR', 'Res_Direct', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"]]
    
    #Distribución RD entre Genéricas de Gasto 
    #FINANCIO GG 2.8 EN PRIMER LUGAR (no se usa saldo de balance para cubrir servicio de la deuda)
    #// Información de servicio de la deuda
    df = df.merge(serv_deuda[["PLIEGO", f"SD_{year}_T"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].round()
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].fillna(0)
    
    #Financio SD con Canon_Sobrecanon y regalías Mineras (// deuda financiada con Canon sobre canon y regalias mineras)
    df['C_SC_RM_sd'] =np.nan
        
    # Reemplazar valores en C_SC_RM_sd basados en una condición
    df['C_SC_RM_sd'] = np.where((df[f"SD_{year}_T"] < df['C_SC_RM']), df[f"SD_{year}_T"],
                               np.where(((df['C_SC_RM'].fillna(0) != 0) & (df['C_SC_RM_sd'].isna()) & (df[f"SD_{year}_T"].fillna(0) != 0)), df['C_SC_RM'], df['C_SC_RM_sd']))
    
    #// identificamos deuda pendiente de financiar con otro rubro
    # Ordenar el DataFrame por la columna PLIEGO
    df = df.sort_values(by='PLIEGO')

    # Calcular la suma acumulativa de C_SC_RM_sd para cada grupo de PLIEGO
    df['t2'] = df.groupby('PLIEGO')['C_SC_RM_sd'].transform('sum')

    # Calcular la diferencia entre SD_20x_T y la suma acumulativa t2
    df['SD4'] = df[f"SD_{year}_T"] - df['t2']
    #*2. Financio SD con Otros ( // deuda financiada con otras fuentes (foncan, exoneraciones))
    df = df.assign(OR_sd = np.nan)
    df['OR_sd'] = np.where((df['SD4']<df['OR']), df['SD4'],
                           np.where(((df['OR'].fillna(0)!= 0)&(df['OR_sd'].isna()) & (df['SD4'].fillna(0)!= 0)), df['OR'], df['OR_sd']))      
    #// identificamos deuda pendiente de financiar con otro rubro
    df['t4'] = df.groupby('PLIEGO')['OR_sd'].transform('sum')
    df['SD5'] = df['SD4'] - df['t4']
    #*3. Financio SD con Participación_Renta_Aduana ( // deuda financiada con renta de aduana)
    df = df.assign(RA_sd = np.nan)
    df['RA_sd'] = np.where((df['SD5']<df['RA']), df['SD5'],
                           np.where(((df['RA'].fillna(0)!= 0) & (df['RA_sd'].isna()) & (df['SD5'].fillna(0)!= 0)), df['RA'], df['RA_sd'])) 
    #// identificamos deuda que no se financiarà
    df['t5'] = df.groupby('PLIEGO')['RA_sd'].transform('sum')
    df['SD6'] = df['SD5'] - df['t5']
    
    # Eliminar columnas innecesarias
    df.drop(columns=[ "t2", "t4", "t5", "SD4", "SD5", "SD6"], inplace=True)
    
    variables = ['C_SC_RM_sd', 'OR_sd', 'RA_sd']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
    # Generando ratios de la propuesta para financiar todo el SD (Tesoro) con RD: pago realizado vs recurso total
    df['ratio_t_csrm'] = df['C_SC_RM_sd']/df['C_SC_RM']
    df['ratio_t_or'] = df['OR_sd']/df['OR']
    df['ratio_t_ra'] = df['RA_sd']/df['RA']
    #Ya no se promedio con el ratio de ejecución promedio dado que se decidió priorizar el pago de la deuda 
    #(Uso los ratios totales de la propuesta de DGETP no los finales promedio.)
    # Calcular las variables ajustadas
    df['C_SC_RM_a'] = round(df['C_SC_RM'] * df['ratio_t_csrm'])
    df['C_SC_RM_a'].fillna(0, inplace=True)

    df['OR_a'] = round(df['OR'] * df['ratio_t_or'])
    df['OR_a'].fillna(0, inplace=True)

    df['RA_a'] = round(df['RA'] * df['ratio_t_ra'])
    df['RA_a'].fillna(0, inplace=True)

    # Calcular SD_ax
    df[f"SD_a{year}"] = df[["C_SC_RM_a", "OR_a","RA_a"]].sum(axis=1) 

    # Eliminar las columnas especificadas
    columns_to_drop = ['C_SC_RM_sd', 'OR_sd', 'RA_sd', 'ratio_t_csrm', 'ratio_t_or', 'ratio_t_ra']
    df.drop(columns=columns_to_drop, inplace=True)
    
    ######################*Para appender a la otrs GGs
    # Agrupar y sumar las variables por PLIEGO
    df = df.groupby('PLIEGO').agg({'C_SC_RM_a': 'sum', 'OR_a': 'sum', 'RA_a': 'sum', f"SD_a{year}": 'sum'}).reset_index()

    # Renombrar columnas
    df.rename(columns={'C_SC_RM_a': 'PR_C_SC_RM', 'OR_a': 'PR_OR', 'RA_a': 'PR_RA', f"SD_a{year}": 'MONTOSD_RD'}, inplace=True)

    # Agregar columnas GENERICA, CATEGORIA_GASTO, SECTOR, FUENTE y RUBRO
    df['GENERICA'] = "8. SERVICIO DE LA DEUDA PUBLICA"
    df['CATEGORIA_GTO'] = "7. SERVICIO DE LA DEUDA"
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES"
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"

    # Eliminar filas donde MONTOSD_RD es igual a 0
    df = df[df['MONTOSD_RD'].fillna(0)!= 0].reset_index(drop=True)
    return df

In [47]:
SDT_2025=PIASBF(PIA_2025_bd24,2025)
SDT_2025.info()
SDT_2025.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          15 non-null     object 
 1   PR_C_SC_RM      15 non-null     float64
 2   PR_OR           15 non-null     float64
 3   PR_RA           15 non-null     float64
 4   MONTOSD_RD      15 non-null     float64
 5   GENERICA        15 non-null     object 
 6   CATEGORIA_GTO   15 non-null     object 
 7   SECTOR          15 non-null     object 
 8   FUENTE          15 non-null     object 
 9   RUBRO           15 non-null     object 
 10  NIVEL_GOBIERNO  15 non-null     object 
dtypes: float64(4), object(7)
memory usage: 1.4+ KB


,PLIEGO,PR_C_SC_RM,PR_OR,PR_RA,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO
0,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6435953.0,0.0,0.0,6435953.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
1,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,168751467.0,0.0,0.0,168751467.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
2,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,27253482.0,0.0,0.0,27253482.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
3,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,33896775.0,0.0,0.0,33896775.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
4,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,8522051.0,0.0,0.0,8522051.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
5,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,9143347.0,0.0,0.0,9143347.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
6,453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO,9266852.0,0.0,0.0,9266852.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
7,455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQ...,2510672.0,0.0,0.0,2510672.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
8,456. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PASCO,4400574.0,0.0,0.0,4400574.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
9,457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA,31142055.0,0.0,0.0,31142055.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES


In [48]:
SDT_2026=PIASBF(PIA_2026_bd24,2026)
SDT_2026.info()
SDT_2026.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          16 non-null     object 
 1   PR_C_SC_RM      16 non-null     float64
 2   PR_OR           16 non-null     float64
 3   PR_RA           16 non-null     float64
 4   MONTOSD_RD      16 non-null     float64
 5   GENERICA        16 non-null     object 
 6   CATEGORIA_GTO   16 non-null     object 
 7   SECTOR          16 non-null     object 
 8   FUENTE          16 non-null     object 
 9   RUBRO           16 non-null     object 
 10  NIVEL_GOBIERNO  16 non-null     object 
dtypes: float64(4), object(7)
memory usage: 1.5+ KB


,PLIEGO,PR_C_SC_RM,PR_OR,PR_RA,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO
0,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,7323825.0,0.0,0.0,7323825.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
1,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,156538489.0,0.0,0.0,156538489.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
2,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,28844125.0,0.0,0.0,28844125.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
3,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,32436416.0,0.0,0.0,32436416.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
4,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,4787244.0,0.0,0.0,4787244.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
5,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,1810681.0,0.0,0.0,1810681.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
6,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,33612243.0,0.0,0.0,33612243.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
7,453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO,127658.0,0.0,0.0,127658.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
8,455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQ...,2508183.0,0.0,0.0,2508183.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
9,456. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PASCO,4062068.0,0.0,0.0,4062068.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES


In [49]:
SDT_2027=PIASBF(PIA_2027_bd24,2027)
SDT_2027.info()
SDT_2027.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          14 non-null     object 
 1   PR_C_SC_RM      14 non-null     float64
 2   PR_OR           14 non-null     float64
 3   PR_RA           14 non-null     float64
 4   MONTOSD_RD      14 non-null     float64
 5   GENERICA        14 non-null     object 
 6   CATEGORIA_GTO   14 non-null     object 
 7   SECTOR          14 non-null     object 
 8   FUENTE          14 non-null     object 
 9   RUBRO           14 non-null     object 
 10  NIVEL_GOBIERNO  14 non-null     object 
dtypes: float64(4), object(7)
memory usage: 1.3+ KB


,PLIEGO,PR_C_SC_RM,PR_OR,PR_RA,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO
0,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,19612855.0,0.0,0.0,19612855.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
1,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,55523384.0,0.0,0.0,55523384.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
2,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,23878471.0,0.0,0.0,23878471.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
3,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,31559441.0,0.0,0.0,31559441.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
4,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,18669586.0,0.0,0.0,18669586.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
5,453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO,127658.0,0.0,0.0,127658.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
6,455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQ...,2505694.0,0.0,0.0,2505694.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
7,456. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PASCO,3177425.0,0.0,0.0,3177425.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
8,457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA,683421.0,0.0,0.0,683421.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES
9,458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO,115700.0,0.0,0.0,115700.0,8. SERVICIO DE LA DEUDA PUBLICA,7. SERVICIO DE LA DEUDA,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. GOBIERNOS REGIONALES


### PIA reducido (menos los recursos para la GG 2.8)

In [50]:
PIA_2025_bd24= PIA_2025_base24.copy()
PIA_2026_bd24= PIA_2026_base24.copy()
PIA_2027_bd24= PIA_2027_base24.copy()

In [51]:
def PIASBF1(df,year):
    
    variables = ['Canon_Sobrecanon', 'Regalias_Mineras', 'Regalias_FOCAM', 'Participación_Elim_Exo', 'Participación_Renta_Aduana', 'Regalias_Contractuales']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
        
    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['C_SC_RM'] = df[["Canon_Sobrecanon", "Regalias_Mineras"]].sum(axis=1) 
    df['OR'] = df[["Regalias_FOCAM", "Participación_Elim_Exo", "Regalias_Contractuales" ]].sum(axis=1)
    df['RA'] = df['Participación_Renta_Aduana']
    
    # Eliminar columnas innecesarias
    df.drop(columns=['Regalias_Mineras', "Regalias_Contractuales", "Regalias_FOCAM", "Participación_Renta_Aduana", 
                     "Participación_Elim_Exo", "Canon_Sobrecanon"], inplace=True)
    df = df[['AÑO', 'PLIEGO', 'C_SC_RM', 'RA', 'OR', 'Res_Direct', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"]]
    
    #Distribución RD entre Genéricas de Gasto 
    #FINANCIO GG 2.8 EN PRIMER LUGAR (no se usa saldo de balance para cubrir servicio de la deuda)
    #// Información de servicio de la deuda
    df = df.merge(serv_deuda[["PLIEGO", f"SD_{year}_T"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].round()
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].fillna(0)
    
    #Financio SD con Canon_Sobrecanon y regalías Mineras (// deuda financiada con Canon sobre canon y regalias mineras)
    df['C_SC_RM_sd'] =np.nan
        
    # Reemplazar valores en C_SC_RM_sd basados en una condición
    df['C_SC_RM_sd'] = np.where((df[f"SD_{year}_T"] < df['C_SC_RM']), df[f"SD_{year}_T"],
                               np.where(((df['C_SC_RM'] != 0) & (df['C_SC_RM_sd'].isna()) & (df[f"SD_{year}_T"] != 0)), df['C_SC_RM'], df['C_SC_RM_sd']))
    
    #// identificamos deuda pendiente de financiar con otro rubro
    # Ordenar el DataFrame por la columna PLIEGO
    df = df.sort_values(by='PLIEGO')

    # Calcular la suma acumulativa de C_SC_RM_sd para cada grupo de PLIEGO
    df['t2'] = df.groupby('PLIEGO')['C_SC_RM_sd'].transform('sum')

    # Calcular la diferencia entre SD_20x_T y la suma acumulativa t2
    df['SD4'] = df[f"SD_{year}_T"] - df['t2']
    #*2. Financio SD con Otros ( // deuda financiada con otras fuentes (foncan, exoneraciones))
    df = df.assign(OR_sd = np.nan)
    df['OR_sd'] = np.where((df['SD4']<df['OR']), df['SD4'],
                           np.where(((df['OR'] != 0) & (df['OR_sd'].isna()) & (df['SD4']!= 0)), df['OR'], df['OR_sd']))      
    #// identificamos deuda pendiente de financiar con otro rubro
    df['t4'] = df.groupby('PLIEGO')['OR_sd'].transform('sum')
    df['SD5'] = df['SD4'] - df['t4']
    #*3. Financio SD con Participación_Renta_Aduana ( // deuda financiada con renta de aduana)
    df = df.assign(RA_sd = np.nan)
    df['RA_sd'] = np.where((df['SD5']<df['RA']), df['SD5'],
                           np.where(((df['RA'].fillna(0) != 0) & (df['RA_sd'].isna()) & (df['SD5'].fillna(0)!= 0)), df['RA'], df['RA_sd'])) 
    #// identificamos deuda que no se financiarà
    df['t5'] = df.groupby('PLIEGO')['RA_sd'].transform('sum')
    df['SD6'] = df['SD5'] - df['t5']
    
    # Eliminar columnas innecesarias
    df.drop(columns=[ "t2", "t4", "t5", "SD4", "SD5", "SD6"], inplace=True)
    
    variables = ['C_SC_RM_sd', 'OR_sd', 'RA_sd']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
    # Generando ratios de la propuesta para financiar todo el SD (Tesoro) con RD: pago realizado vs recurso total
    df['ratio_t_csrm'] = df['C_SC_RM_sd']/df['C_SC_RM']
    df['ratio_t_or'] = df['OR_sd']/df['OR']
    df['ratio_t_ra'] = df['RA_sd']/df['RA']
    #Ya no se promedio con el ratio de ejecución promedio dado que se decidió priorizar el pago de la deuda 
    #(Uso los ratios totales de la propuesta de DGETP no los finales promedio.)
    # Calcular las variables ajustadas
    df['C_SC_RM_a'] = round(df['C_SC_RM'] * df['ratio_t_csrm'])
    df['C_SC_RM_a'].fillna(0, inplace=True)

    df['OR_a'] = round(df['OR'] * df['ratio_t_or'])
    df['OR_a'].fillna(0, inplace=True)

    df['RA_a'] = round(df['RA'] * df['ratio_t_ra'])
    df['RA_a'].fillna(0, inplace=True)

    # Calcular SD_ax
    df[f"SD_a{year}"] = df[["C_SC_RM_a", "OR_a","RA_a"]].sum(axis=1) 

    # Eliminar las columnas especificadas
    columns_to_drop = ['C_SC_RM_sd', 'OR_sd', 'RA_sd', 'ratio_t_csrm', 'ratio_t_or', 'ratio_t_ra']
    df.drop(columns=columns_to_drop, inplace=True)
    
    ########Guardamos el PIA reducido (menos los recursos para la GG 2.8)
    
    # Reemplazar valores en f"PIA_BASE_{year}"
    df[f"PIA_BASE_{year}"] = df[f"PIA_BASE_{year}"] - df[f"SD_a{year}"]

    # Reemplazar valores en OR, RA y C_SC_RM
    df['OR'] = df['OR'] - df['OR_a']
    df['RA'] = df['RA'] - df['RA_a']
    df['C_SC_RM'] = df['C_SC_RM'] - df['C_SC_RM_a']

    # Renombrar columnas
    df.rename(columns={'OR': 'Otros_Rec', 'RA': 'R_A'}, inplace=True)

    # Eliminar columnas especificadas
    columns_to_drop = [f"SD_a{year}", 'OR_a', 'RA_a', 'C_SC_RM_a', f"SD_{year}_T"]
    df.drop(columns=columns_to_drop, inplace=True)
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES"
    
    return df

In [52]:
PIA_2025=PIASBF1(PIA_2025_bd24,2025)
PIA_2025.info()
PIA_2025.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             26 non-null     int64  
 1   PLIEGO          26 non-null     object 
 2   C_SC_RM         26 non-null     float64
 3   R_A             26 non-null     float64
 4   Otros_Rec       26 non-null     float64
 5   Res_Direct      26 non-null     int64  
 6   SB_2025         26 non-null     float64
 7   PMG_2025_gg3_1  26 non-null     float64
 8   PMG_2025_gg6_2  26 non-null     float64
 9   PIA_BASE_2025   26 non-null     float64
 10  NIVEL_GOBIERNO  26 non-null     object 
 11  SECTOR          26 non-null     object 
 12  FUENTE          26 non-null     object 
 13  RUBRO           26 non-null     object 
dtypes: float64(7), int64(2), object(5)
memory usage: 3.0+ KB


,AÑO,PLIEGO,C_SC_RM,R_A,Otros_Rec,Res_Direct,SB_2025,PMG_2025_gg3_1,PMG_2025_gg6_2,PIA_BASE_2025,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2025,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,4539.0,0.0,11457628.0,11462167,500.0,0.0,0.0,11462667.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,2025,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,220514363.0,0.0,0.0,220514363,101150000.0,0.0,0.0,321664363.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,2025,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,23548795.0,0.0,40628249.0,70612997,2174700.0,0.0,0.0,66351744.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,2025,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,1505583.0,0.0,0.0,170257050,16000000.0,4367603.0,0.0,17505583.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,2025,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,10392377.0,0.0,41125059.0,51517436,6044986.0,2808026.0,20000.0,57562422.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
5,2025,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2263977.0,0.0,0.0,29517459,0.0,0.0,0.0,2263977.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
6,2025,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,585478875.0,0.0,0.0,619375650,43934200.0,0.0,0.0,629413075.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
7,2025,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,17345062.0,0.0,22611726.0,39956788,100000.0,0.0,0.0,40056788.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
8,2025,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,328230.0,0.0,0.0,328230,1277834.0,268700.0,919500.0,1606064.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
9,2025,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,115302513.0,0.0,34927453.0,158752017,18000000.0,266000.0,0.0,168229966.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


In [53]:
PIA_2026=PIASBF1(PIA_2026_bd24,2026)
PIA_2026.info()
PIA_2026.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             26 non-null     int64  
 1   PLIEGO          26 non-null     object 
 2   C_SC_RM         26 non-null     float64
 3   R_A             26 non-null     float64
 4   Otros_Rec       26 non-null     float64
 5   Res_Direct      26 non-null     int64  
 6   SB_2026         26 non-null     float64
 7   PMG_2026_gg3_1  26 non-null     float64
 8   PMG_2026_gg6_2  26 non-null     float64
 9   PIA_BASE_2026   26 non-null     float64
 10  NIVEL_GOBIERNO  26 non-null     object 
 11  SECTOR          26 non-null     object 
 12  FUENTE          26 non-null     object 
 13  RUBRO           26 non-null     object 
dtypes: float64(7), int64(2), object(5)
memory usage: 3.0+ KB


,AÑO,PLIEGO,C_SC_RM,R_A,Otros_Rec,Res_Direct,SB_2026,PMG_2026_gg3_1,PMG_2026_gg6_2,PIA_BASE_2026,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2026,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,4539.0,0.0,11686781.0,11691320,500.0,0.0,0.0,11691820.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,2026,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,211761392.0,0.0,0.0,211761392,101150000.0,0.0,0.0,312911392.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,2026,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,21581685.0,0.0,37470685.0,66376195,2555000.0,0.0,0.0,61607370.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,2026,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,0.0,0.0,0.0,156538489,16000000.0,1367603.0,0.0,16000000.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,2026,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,9953156.0,0.0,39361870.0,49315026,6212450.0,3802194.0,0.0,55527476.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
5,2026,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,0.0,0.0,0.0,28844125,0.0,0.0,0.0,0.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
6,2026,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,575031922.0,0.0,0.0,607468338,41706169.0,0.0,0.0,616738091.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
7,2026,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,19156594.0,0.0,21642275.0,40798869,100000.0,0.0,0.0,40898869.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
8,2026,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,10198.0,0.0,0.0,10198,1182954.0,228820.0,860000.0,1193152.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
9,2026,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,108888161.0,0.0,33429980.0,147105385,17000000.0,268000.0,0.0,159318141.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


In [54]:
PIA_2027=PIASBF1(PIA_2027_bd24,2027)
PIA_2027.info()
PIA_2027.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             26 non-null     int64  
 1   PLIEGO          26 non-null     object 
 2   C_SC_RM         26 non-null     float64
 3   R_A             26 non-null     float64
 4   Otros_Rec       26 non-null     float64
 5   Res_Direct      26 non-null     int64  
 6   SB_2027         26 non-null     float64
 7   PMG_2027_gg3_1  26 non-null     float64
 8   PMG_2027_gg6_2  26 non-null     float64
 9   PIA_BASE_2027   26 non-null     float64
 10  NIVEL_GOBIERNO  26 non-null     object 
 11  SECTOR          26 non-null     object 
 12  FUENTE          26 non-null     object 
 13  RUBRO           26 non-null     object 
dtypes: float64(7), int64(2), object(5)
memory usage: 3.0+ KB


,AÑO,PLIEGO,C_SC_RM,R_A,Otros_Rec,Res_Direct,SB_2027,PMG_2027_gg3_1,PMG_2027_gg6_2,PIA_BASE_2027,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2027,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,4539.0,0.0,11920517.0,11925056,500.0,0.0,0.0,11925556.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
1,2027,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,205623228.0,0.0,0.0,205623228,101150000.0,0.0,0.0,306773228.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
2,2027,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,8420861.0,0.0,35065736.0,63099452,3065000.0,0.0,0.0,46551597.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
3,2027,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,94206195.0,0.0,0.0,149729579,16000000.0,0.0,0.0,110206195.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
4,2027,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,8715726.0,0.0,37454250.0,46169976,6618876.0,3064000.0,0.0,52788852.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
5,2027,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2649471.0,0.0,0.0,26527942,0.0,0.0,0.0,2649471.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
6,2027,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,558783933.0,0.0,0.0,590343374,42126787.0,0.0,0.0,600910720.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
7,2027,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,15628897.0,0.0,20593411.0,36222308,100000.0,0.0,0.0,36322308.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
8,2027,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,215901.0,0.0,0.0,215901,1178334.0,227200.0,860000.0,1394235.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."
9,2027,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,107949733.0,0.0,31809840.0,139759573,16000000.0,270000.0,0.0,155759573.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU..."


In [55]:
PIA_2025_1 = PIA_2025.copy()
PIA_2026_1 = PIA_2026.copy()
PIA_2027_1 = PIA_2027.copy()
SDT_2025_1 = SDT_2025.copy()
SDT_2026_1 = SDT_2026.copy()
SDT_2027_1 = SDT_2027.copy()
PIA_2025_base241= PIA_2025_base24.copy()
PIA_2026_base241=PIA_2026_base24.copy()
PIA_2027_base241=PIA_2027_base24.copy()

### FINANCIO OTRAS GGs (Cruce con ratios para la distribución con las otras GG de gasto - GENERANDO PROPUESTA CON RATIOS PROMEDIOS )

In [ ]:
#Cruce con ratios para la distribución con las otras GG de gasto* (GENERANDO PROPUESTA CON RATIOS PROMEDIOS)

def PropyRatios(df, df1, df2, year):
    
    ratios_sb_c1= ratios_sb_c.copy()
    ratios_sb_c1.drop(columns=['mean_ratio_C_SC_RM', 'mean_ratio_R_Aduana', 'comp', 'compra'], inplace=True)

    # aplicatr merge bases PIA_202* con ratios_sb_c1
    df = pd.merge(ratios_sb_c1, df[["PLIEGO", 'C_SC_RM', 'Otros_Rec', 'R_A', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"]], on="PLIEGO", how="outer", validate = 'm:1')
    postmerge(df)

    #  Ordenar columnas 
    df = df.reindex(columns=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA', 'ratio_f_c', 'ratio_f_ra', 'ratio_sb_fed',
                                               'C_SC_RM', 'Otros_Rec', 'R_A',  f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"])

    # Calcular nuevas columnas 
    df['PR_C_SC_RM'] = round(df['ratio_f_c'] * df['C_SC_RM'])
    df['PR_OR'] = df['Otros_Rec'].where(df['GENERICA'].str[0] == '6')
    df['PR_RA'] = round(df['ratio_f_ra'] * df['R_A'])
    df[f"PR_SB_{year}"] = np.nan
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)>1)&(df['GENERICA'].str[0] == '3'), round(df[f"PMG_{year}_gg3_1"]), df[f"PR_SB_{year}"])
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)>1)&(df['GENERICA'].str[0] == '6'), round(df[f"SB_{year}"] - df[f"PMG_{year}_gg3_1"]), df[f"PR_SB_{year}"])
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)<=0)&(df['GENERICA'].str[0] == '3'), round(df['ratio_sb_fed'] * df[f"SB_{year}"]), df[f"PR_SB_{year}"])
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)<=0)&(df['GENERICA'].str[0] == '6'), round((1 - df['ratio_sb_fed']) * df[ f"SB_{year}"]), df[f"PR_SB_{year}"])
   
    #df[f"PR_SB_{year}"] = round(df['ratio_sb_fed'] * df[f"SB_{year}"]).where(df['GENERICA'].str[0] == '3')
    #df.loc[df['GENERICA'].str[0] == '6', f"PR_SB_{year}"] = round((1 - df['ratio_sb_fed']) * df[ f"SB_{year}"])

    df['PROP_RATIOS'] = df[['PR_C_SC_RM', 'PR_OR', 'PR_RA',  f"PR_SB_{year}"]].sum(axis=1)
    df['dif'] = df[f"PIA_BASE_{year}"] - df['PROP_RATIOS']
    df['tot_p'] = df[['C_SC_RM', 'Otros_Rec', 'R_A', f"SB_{year}"]].sum(axis=1)
    df['dif2'] = df[f"PIA_BASE_{year}"] - df['tot_p']

    df.drop(columns=['tot_p', 'dif2'], inplace=True)
    df.rename(columns=lambda x: x.upper(), inplace=True)

    # Concatenar data( // incluimos servicio de la deuda)
    df = pd.concat([df, df1], ignore_index=True)

    # Reemplazar valores missing
    df['PROP_RATIOS'].fillna(0, inplace=True)
    df['MONTOSD_RD'].fillna(0, inplace=True)

    df['T'] = df[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1)

    # Eliminar filas que contengan T == 0
    df = df[df['T'].fillna(0) != 0]
    
    #*Dando formato a propuesta final
    # Dando formato a propuesta final
    df['PROP_RATIOS'].fillna(0, inplace=True)
    df['MONTOSD_RD'].fillna(0, inplace=True)
    df['MONTOF_RD'] = df[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1)
    df['NIVEL_GOBIERNO'].replace({'GOBIERNOS REGIONALES': '2. GOBIERNOS REGIONALES'}, inplace=True)
    
    df['CATEGORIA_GTO'] = np.where(df['CATEGORIA_GTO'].str.contains('5'),'5. GASTOS CORRIENTES',
                                     np.where(df['CATEGORIA_GTO'].str.contains('6'), '6. GASTOS DE CAPITAL', df['CATEGORIA_GTO']))
    
    # Ajustes de redondeo
    df['PROP'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO'])['MONTOF_RD'].transform('sum')
    df.drop(columns=[f"PIA_BASE_{year}"], inplace=True)

    df = pd.merge(df, df2, on='PLIEGO', how="outer", validate = 'm:1')
    postmerge(df)
    df['difb'] = df[f"PIA_BASE_{year}"] - df['PROP']
    df['MONTOF_RD'].fillna(0, inplace=True)
    df['difb'].fillna(0, inplace=True)
    
    df['MONTOF_RD'] = np.where((df['difb'] < 0) & (df['GENERICA'].str[0] == '3'), 
                               df[['MONTOF_RD', 'difb']].sum(axis=1),
                               np.where((df['difb'] > 0) & (df['GENERICA'].str[0] == '6'), df[['MONTOF_RD', 'difb']].sum(axis=1), 
                                        df['MONTOF_RD']))
    
    df.drop(columns=['difb'], inplace=True)
    #*c. Ajustes de distribución por marco legal (c1. REASIGNANDO EL MONTO DE LA GG 2.4)
    df['Monto_GG4_CO'] = np.where(((df['GENERICA'].str[0] == '4') & (df['CATEGORIA_GTO'].str[0] == '5')), df['MONTOF_RD'], np.nan)    
    df['Monto_GG4_CA'] = np.where(((df['GENERICA'].str[0] == '4') & (df['CATEGORIA_GTO'].str[0] == '6')), df['MONTOF_RD'], np.nan)    
    # Calcular sumas por grupos
    df['PLIEGO_GG24_CO'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'])['Monto_GG4_CO'].transform('sum')
    df['PLIEGO_GG24_CA'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'])['Monto_GG4_CA'].transform('sum')

    # Reemplazar valores nulos con 0
    df['PLIEGO_GG24_CO'].fillna(0, inplace=True)
    df['PLIEGO_GG24_CA'].fillna(0, inplace=True)

    # Actualizar MONTOF_RD según condiciones
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '3', df[['MONTOF_RD', 'PLIEGO_GG24_CO']].sum(axis=1), df['MONTOF_RD'])
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '6', df[['MONTOF_RD', 'PLIEGO_GG24_CA']].sum(axis=1), df['MONTOF_RD'])
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '4', df['MONTOF_RD'], df['MONTOF_RD'])
    
    #*c2. REASIGNANDO EL MONTO DE LA GG 2.5 (CALLAO Y LORETO ) - Loreto siempre fue la única en tener Capital - por eso se hace ajuste para Callao
    df['Monto_GG5_CO'] = np.where(((df['GENERICA'].str[0] == '5') & (df['CATEGORIA_GTO'].str[0] == '5')), df['MONTOF_RD'], np.nan)
    df['Monto_GG5_CA'] = np.where(((df['GENERICA'].str[0] == '5') & (df['CATEGORIA_GTO'].str[0] == '6')), df['MONTOF_RD'], np.nan)   
    
    # Calcular sumas por grupos
    df['PLIEGO_GG25_CO'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA' ])['Monto_GG5_CO'].transform('sum')
    df['PLIEGO_GG25_CA'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA' ])['Monto_GG5_CA'].transform('sum')
    
    # Reemplazar valores nulos con 0
    df['PLIEGO_GG25_CO'].fillna(0, inplace=True)
    df['PLIEGO_GG25_CA'].fillna(0, inplace=True)

    
    # Reemplazar MONTOF_RD según la primera condición
    df['MONTOF_RD'] = np.where((df['GENERICA'].str.contains('3') & ~(df['PLIEGO'].str.contains('464'))), df[['MONTOF_RD', 'PLIEGO_GG25_CO']].sum(axis=1),
                               np.where((df['GENERICA'].str.contains('5') & df['CATEGORIA_GTO'].str.contains('5') & ~(df['PLIEGO'].str.contains('464'))), 0, df['MONTOF_RD']))

    # Reemplazar GENERICA según la tercera condición
    df.loc[(df['GENERICA'] == '7. ADQUISICION DE ACTIVOS FINANCIEROS') & (df['PLIEGO'] == '453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO'), 'GENERICA'] = '6. ADQUISICION DE ACTIVOS NO FINANCIEROS'
    #* c3. VERIFICANDO LA PARTICIPACION DEL GASTO CORRIENTE ( (20% DEL CANON, SOBRECANON Y RM PARA MANTENIMIENTO))
    
    # Verificando la participación del gasto corriente
    df['Totcanon'] = df.groupby('PLIEGO')['PR_C_SC_RM'].transform('sum')
    df['ratio_canon'] = df['PR_C_SC_RM'] / df['Totcanon']
    df['EXCESO_GG3'] = np.where((df['ratio_canon'] > 0.2) & df['GENERICA'].str.contains('3'), np.round(df['ratio_canon']- 0.2, 4), np.nan)
    df['MOD'] = np.round(df['EXCESO_GG3'] * df[f"PIA_BASE_{year}"])    
    df['TotMOD'] = df.groupby('PLIEGO')['MOD'].transform('sum')
    df['MONTOF_RD'].fillna(0, inplace=True)
    df['TotMOD'].fillna(0, inplace=True)
    df['MONTOF_RD'] = np.where(df['GENERICA'].str.contains('6') & (~df['PLIEGO'].str.contains('CALLAO')),df[['MONTOF_RD', 'TotMOD']].sum(axis=1),
                               np.where(df['GENERICA'].str.contains('3') & (~df['PLIEGO'].str.contains('CALLAO')), df['MONTOF_RD'] - df['TotMOD'], df['MONTOF_RD']))
    df['ratio_canon2'] = df['PR_C_SC_RM'] / df['Totcanon']
    df['EXCESO_GG3_2'] = np.where((df['ratio_canon2'] > 0.2) & df['GENERICA'].str.contains('3'), np.round(df['ratio_canon2'] - 0.2, 4), np.nan)
    
    columnas = list(df.columns)
    df = df[["NIVEL_GOBIERNO"] + columnas[:31] + columnas[32:]]
    df.drop(columns=['NIVEL_GOBIERNO'], inplace=True)
    df['NIVEL_GOBIERNO'] = '2. GOBIERNOS REGIONALES'
    df['RUBRO'] = '18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES'
    
    return df

In [ ]:
PropyRatios25=PropyRatios(PIA_2025_1,SDT_2025_1, PIA_2025_base241, 2025)
PropyRatios25.info()
PropyRatios25.head(100)

In [ ]:
PropyRatios26=PropyRatios(PIA_2026_1,SDT_2026_1, PIA_2026_base241, 2026)
PropyRatios26.info()
PropyRatios26.head(100)

In [ ]:
PropyRatios27=PropyRatios(PIA_2027_1,SDT_2027_1, PIA_2027_base241, 2027)
PropyRatios27.info()
PropyRatios27.head(100)

In [ ]:
PropyRatios25_1 = PropyRatios25.copy()
PropyRatios26_1 = PropyRatios26.copy()
PropyRatios27_1 = PropyRatios27.copy()

In [ ]:
def MONTOF_RD(data, year):
    
    #*Formato Pliego por tipo de recurso
    data.rename(columns=lambda x: x.upper(), inplace=True)
    # Calcular MONTOI_RD
    data['MONTOI_RD'] = data[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1) 

    # Calcular AJUSTES
    data['AJUSTES'] = data['MONTOF_RD'] - data['MONTOI_RD']

    # Agrupar y colapsar
    data = data.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA']).agg({'PR_C_SC_RM': 'sum', 'PR_OR': 'sum',
                                                                                                                     'PR_RA': 'sum', f"PR_SB_{year}": 'sum',
                                                                                                                     'AJUSTES': 'sum', 'MONTOI_RD': 'sum',
                                                                                                                     'MONTOF_RD': 'sum'}).reset_index()

    # Reemplazar PR_C_SC_RM según la condición
    data['PR_C_SC_RM'] = data[['PR_C_SC_RM', 'AJUSTES']].sum(axis=1) 

    # Eliminar filas donde MONTOI_RD es 0
    data = data[data['MONTOI_RD']. fillna(0) != 0]
    
    data['PR_RA'].fillna(0, inplace=True)
    data['AJUSTES'].fillna(0, inplace=True)
    
    # Reemplazar PR_RA según la condición
    data['PR_RA'] = np.where(data['PR_C_SC_RM'] < 0, data[['PR_RA', 'AJUSTES']].sum(axis=1), data['PR_RA'])

    # Reemplazar valores negativos de PR_C_SC_RM con 0
    data['PR_C_SC_RM'] = np.where(data['PR_C_SC_RM'] < 0, 0, data['PR_C_SC_RM'])

    # Agregar columna AÑO
    data['AÑO'] = year

    # Ordenar columnas
    data = data[['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'AÑO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA', 'MONTOI_RD', 'AJUSTES', 'MONTOF_RD', 'PR_C_SC_RM', 'PR_OR', 'PR_RA', f"PR_SB_{year}"]]

    # Renombrar columnas
    data.rename(columns={'PR_C_SC_RM': f"PR_C_SC_RM{year}", 'PR_OR': f'PR_OR{year}', 'PR_RA': f'PR_RA{year}', 'MONTOF_RD': f'PMG_{year}'}, inplace=True)

    return data

In [ ]:
MONTOF_RD_25=MONTOF_RD(PropyRatios25_1, 2025)
MONTOF_RD_25.info()
MONTOF_RD_25.head(100)

In [ ]:
MONTOF_RD_26=MONTOF_RD(PropyRatios26_1, 2026)
MONTOF_RD_26.info()
MONTOF_RD_26.head(100)

In [ ]:
MONTOF_RD_27=MONTOF_RD(PropyRatios27_1, 2027)
MONTOF_RD_27.info()
MONTOF_RD_27.head(100)

In [ ]:
MONTOF_RD_25_1=MONTOF_RD_25.copy()
MONTOF_RD_26_1=MONTOF_RD_26.copy()
MONTOF_RD_27_1=MONTOF_RD_27.copy()

In [ ]:
# Realizar la primera fusión m:m
Propuesta = pd.merge(MONTOF_RD_25_1, MONTOF_RD_26_1, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'], how='outer')
postmerge(Propuesta)

# Realizar la segunda fusión m:m
Propuesta_RD_GG = pd.merge(Propuesta, MONTOF_RD_27_1, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'], how='outer')
postmerge(Propuesta_RD_GG)

# Seleccionar columnas necesarias
Propuesta_RD_GG = Propuesta_RD_GG[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA',
                                     'PMG_2025', 'PR_C_SC_RM2025', 'PR_OR2025', 'PR_RA2025', 'PR_SB_2025',
                                     'PMG_2026', 'PR_C_SC_RM2026', 'PR_OR2026', 'PR_RA2026', 'PR_SB_2026',
                                     'PMG_2027', 'PR_C_SC_RM2027', 'PR_OR2027', 'PR_RA2027', 'PR_SB_2027']]
# 1 FLUJO 2 ES SALDO
# Calcular PMG2025, PMG2026 y PMG2027
Propuesta_RD_GG['PMG20251'] = Propuesta_RD_GG[["PR_C_SC_RM2025", "PR_OR2025", "PR_RA2025" ]].sum(axis=1)
Propuesta_RD_GG['PMG20261'] = Propuesta_RD_GG[["PR_C_SC_RM2026", "PR_OR2026", "PR_RA2026" ]].sum(axis=1)
Propuesta_RD_GG['PMG20271'] = Propuesta_RD_GG[["PR_C_SC_RM2027", "PR_OR2027", "PR_RA2027" ]].sum(axis=1)

# Renombrar columnas PR_SB25, PR_SB26 y PR_SB27
Propuesta_RD_GG.rename(columns={'PR_SB_2025': 'PMG20252', 'PR_SB_2026': 'PMG20262', 'PR_SB_2027': 'PMG20272'}, inplace=True)

# Seleccionar columnas requeridas
Propuesta_RD_GG = Propuesta_RD_GG[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA',
                                 'PMG20251', 'PMG20261', 'PMG20271', 'PMG20252', 'PMG20262', 'PMG20272']]

# Cambiar formato de ancho a largo (reshape)2025
Propuesta_RD_GG25 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20251', 'PMG20252'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2025')
Propuesta_RD_GG25['TIPO_GEN_ING'] = Propuesta_RD_GG25['TIPO_GEN_ING'].str[-1]

# Cambiar formato de ancho a largo (reshape)
Propuesta_RD_GG26 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20261', 'PMG20262'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2026')
Propuesta_RD_GG26['TIPO_GEN_ING'] = Propuesta_RD_GG26['TIPO_GEN_ING'].str[-1]

# Cambiar formato de ancho a largo (reshape)
Propuesta_RD_GG27 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20271', 'PMG20272'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2027')
Propuesta_RD_GG27['TIPO_GEN_ING'] = Propuesta_RD_GG27['TIPO_GEN_ING'].str[-1]

Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GG25, Propuesta_RD_GG26, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA','TIPO_GEN_ING'], how='outer')
postmerge(Propuesta_RD_GGG1)
Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GGG1, Propuesta_RD_GG27, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA','TIPO_GEN_ING'], how='outer')
postmerge(Propuesta_RD_GGG1)

Propuesta_RD_GGG1 = Propuesta_RD_GGG1[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA',
                                     'PMG_2025', 'PMG_2026', 'PMG_2027']]

Propuesta_RD_GGG1['TIPO_GEN_ING'].replace({'1': '2. OTRAS GENÉRICAS', '2':'1. SALDOS DE BALANCE'}, inplace=True)

Propuesta_RD_GGG1 = Propuesta_RD_GGG1[~((Propuesta_RD_GGG1['PMG_2025'].fillna(0)==0)&(Propuesta_RD_GGG1['PMG_2026'].fillna(0)==0)&(Propuesta_RD_GGG1['PMG_2027'].fillna(0)==0))]
#Propuesta_RD_GGG1 = Propuesta_RD_GGG1[((Propuesta_RD_GGG1['PMG_2025'].fillna(0)!=0)|(Propuesta_RD_GGG1['PMG_2026'].fillna(0)!=0)|(Propuesta_RD_GGG1['PMG_2027'].fillna(0)!=0))]
Propuesta_RD_GGG1.info()
Propuesta_RD_GGG1.head(768)

In [ ]:
# Realizar correcciones de redondeo
Propuesta_RD_GGG= Propuesta_RD_GGG1.copy()
Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 8, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] + 3, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2025'] - 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] + 2, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] + 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2025'] + 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] - 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] - 2, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 6, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 8, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 4, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVELICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 16, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 11, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 18, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN MARTIN") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 4, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN MARTIN") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 4, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 2, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "461. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCAYALI") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 3, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCAYALI") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 2, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

# Eliminar filas duplicadas en el dataframe
Propuesta_RD_GGG.drop_duplicates(inplace=True)

# Crear indicador de pliegos que superan el 20% a nivel de la GG 2.3.
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO25']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2025'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO26']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2026'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO27']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2027'].transform('sum').astype(float)

Propuesta_RD_GGG ['TOTAL_PLIEGO25']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2025'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_PLIEGO26']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2026'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_PLIEGO27']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2027'].transform('sum').astype(float)

Propuesta_RD_GGG ['IND_GEN_PLIEGO25']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO25'] / Propuesta_RD_GGG ['TOTAL_PLIEGO25'])*100
Propuesta_RD_GGG ['IND_GEN_PLIEGO26']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO26'] / Propuesta_RD_GGG ['TOTAL_PLIEGO26'])*100
Propuesta_RD_GGG ['IND_GEN_PLIEGO27']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO27'] / Propuesta_RD_GGG ['TOTAL_PLIEGO27'])*100

Propuesta_RD_GGG ['ALERTA_SUP20%_25'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO25'] > 20), 'SI', 'NO')
Propuesta_RD_GGG ['ALERTA_SUP20%_26'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO26'] > 20), 'SI', 'NO')
Propuesta_RD_GGG ['ALERTA_SUP20%_27'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO27'] > 20), 'SI', 'NO')
Propuesta_RD_GGG.info()
Propuesta_RD_GGG.head(139)

In [ ]:
seg23 = pd.merge(bd_seg23, dict23, on=['COD_CADENA_GASTO'], how='left')
postmerge(seg23)
seg23 = pd.merge(seg23, dict231, on=['COD_ID'], how='left')
postmerge(seg23)
seg23.rename(columns={'PIA' : 'PIA_2023',
                      'PIM' : 'PIM_2023',
                      'TOTAL_DEVENGADO' : 'DEV_2023',
                      'CATEGORIA': 'CATEGORIA_GTO' }, inplace=True)
seg23= seg23[(seg23['COD_ID'].str[0] == '4') & (seg23['COD_ID'].str[3] == '-') ]
seg23= seg23[seg23['COD_FUENTE_RUBRO'] == 'FR.5.18']
seg23['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
seg23['FUENTE'] =  "5. RECURSOS DETERMINADOS"
seg23['RUBRO'] =  "18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES"
seg23['TIPO_GEN_ING'] =  "2. OTRAS GENÉRICAS"
seg23 = seg23[['NIVEL_GOBIERNO', 'COD_ID', 'SECTOR', 'PLIEGO', 'COD_FUENTE_RUBRO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'COD_CADENA_GASTO', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023']].reset_index(drop=True)
seg23 = seg23.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA']).agg({'PIA_2023':'sum', 'PIM_2023':'sum', 'DEV_2023':'sum'}).reset_index()
seg23.info()
seg23.head(29575)

In [ ]:
seg24 = pd.merge(bd_seg24, dict24, on=['COD_CADENA_GASTO'], how='left')
postmerge(seg24)
seg24 = pd.merge(seg24, dict241, on=['COD_ID'], how='left')
postmerge(seg24)
seg24.rename(columns={'PIA' : 'PIA_2024',
                      'PIM' : 'PIM_2024',
                      'TOTAL_DEVENGADO' : 'DEV_2024',
                      'CATEGORIA': 'CATEGORIA_GTO' }, inplace=True)
seg24= seg24[(seg24['COD_ID'].str[0] == '4') & (seg24['COD_ID'].str[3] == '-') ]
seg24= seg24[seg24['COD_FUENTE_RUBRO'] == 'FR.5.18']
seg24['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
seg24['FUENTE'] =  "5. RECURSOS DETERMINADOS"
seg24['RUBRO'] =  "18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES"
seg24['TIPO_GEN_ING'] =  "2. OTRAS GENÉRICAS"
seg24 = seg24[['NIVEL_GOBIERNO', 'COD_ID', 'SECTOR', 'PLIEGO', 'COD_FUENTE_RUBRO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'COD_CADENA_GASTO', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2024', 'PIM_2024', 'DEV_2024']].reset_index(drop=True)
seg24 = seg24.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA']).agg({'PIA_2024':'sum', 'PIM_2024':'sum', 'DEV_2024':'sum'}).reset_index()
seg24.info()
seg24.head(29575)

In [ ]:
Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GGG, seg23, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA'], how='left')
postmerge(Propuesta_RD_GGG1)
Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GGG1, seg24, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA'], how='left')
postmerge(Propuesta_RD_GGG1)
Propuesta_RD_GGG1=Propuesta_RD_GGG1[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023','PIA_2024', 'PIM_2024', 'DEV_2024', 'PMG_2025','PMG_2026','PMG_2027', 
                                     'TOTAL_GEN_PLIEGO25', 'TOTAL_PLIEGO25', 'IND_GEN_PLIEGO25', 'ALERTA_SUP20%_25', 'TOTAL_GEN_PLIEGO26', 'TOTAL_PLIEGO26', 'IND_GEN_PLIEGO26', 'ALERTA_SUP20%_26', 'TOTAL_GEN_PLIEGO27', 'TOTAL_PLIEGO27', 'IND_GEN_PLIEGO27', 'ALERTA_SUP20%_27']]
Propuesta_RD_GGG1.info()
Propuesta_RD_GGG1.head()

In [ ]:
Propuesta_RD_GGG2 = Propuesta_RD_GGG1.copy()
Propuesta_RD_GGG2 = Propuesta_RD_GGG2[Propuesta_RD_GGG2['GENERICA'] == "3. BIENES Y SERVICIOS" ]
outputFile = os.path.join(r'Y:\PMG 2025-2027\Outputs','Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_CANON_1'+ fecha_corte+ ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    Propuesta_RD_GGG1.to_excel(ew, sheet_name="RD GASTO total", index = False)
    Propuesta_RD_GGG2.to_excel(ew, sheet_name="RD GG2.3", index = False)
#%%
        